# Declare dependancies of the project

In [4]:
:dep itertools
:dep poloto
:dep splines
:dep csv
use itertools::Itertools;
use poloto::build;
use splines::Key;
use std::{
    collections::HashMap,
    fs::File,
    hash::BuildHasher,
    io::{self, BufReader},
};

# Data importing
All data comes from the csv files in ./data and goes from 1990 to 2021

In [5]:
const START_YEAR: usize = 1990;
const END_YEAR: usize = 2021;

In [6]:
fn value_from_year(year: usize, data: &[f64]) -> Option<f64> {
    year.checked_sub(START_YEAR)
        .and_then(|index| data.get(index).copied())
}

In [7]:
fn load_hdi() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/HDI.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(1).and_then(|name| {
                (5..37)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

In [8]:
fn load_gdp() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/GDP.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(0).and_then(|name| {
                (1..33)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

The gini coeficint data is sparse so we'll use splines to interpolate it

In [9]:
fn sample_spline(all: Vec<(usize, Option<f64>)>, spline: splines::Spline<f64, f64>) -> Vec<f64> {
    all.iter()
        .map(|(year, value)| {
            value.map_or_else(
                || spline.clamped_sample((*year + START_YEAR) as f64).unwrap(),
                |value| value,
            )
        })
        .collect_vec()
}

In [10]:
fn load_gini() -> Option<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/Gini.csv").ok()?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Some(
        csv.records()
            .flatten()
            .filter_map(|row| {
                row.get(0).and_then(|name| {
                    let all = (1..33)
                        .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                        .enumerate()
                        .collect_vec();

                    let keys = all
                        .iter()
                        .filter_map(|(y, v)| v.map(|value| (value, (y + START_YEAR) as f64)))
                        .map(|(v, y)| Key::new(y, v, splines::Interpolation::Cosine))
                        .collect_vec();

                    if keys.is_empty() {
                        None
                    } else {
                        // let first = keys[0];
                        // let last = keys[keys.len() - 1];
                        // let spline =
                        //     splines::Spline::from_vec([vec![first], keys, vec![last]].concat());
                        let spline = splines::Spline::from_vec(keys);

                        Some((name.to_string(), sample_spline(all, spline)))
                    }
                })
            })
            .collect::<HashMap<String, Vec<f64>>>(),
    )
}

# Using the data

In [47]:
let gdp_data = load_gdp().unwrap();
let hdi_data = load_hdi().unwrap();
let gini_data = load_gini().unwrap();



## Plotting
First lets plot hdi against gdp

In [88]:
use std::fmt::Debug;
pub struct Matrix<T> {pub data: T}
impl<T: Debug> Matrix<T> {
    pub fn evcxr_display(&self) {
        let mut html = String::new();
        html.push_str("<div>");
          html.push_str(&format!("{:?}", self.data));
        html.push_str("</div>");
        println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
    }
}
let m = Matrix {data: "G"};
m

"G"

In [90]:
let mut data = vec![];
    for (name, hdi_points) in hdi_data.iter() {
        if let Some(gdp_points) = gdp_data.get(name) {
            for point in gdp_points.iter().zip(hdi_points.iter()) {
                data.push(point);
            }
        }
    }

    let plots = poloto::plots!(poloto::build::plot("").scatter(data));

    let svg_data = poloto::frame_build()
        .data(poloto::plots!(
            build::markers(vec![2000., 2030.], [0., 1.]),
            plots
        ))
        .build_and_label(("HDI vs GDP", "HDI", "GDP"))
        .append_to(poloto::header().light_theme())
        .render_string().unwrap();

    Matrix { data : &svg_data }

" \n\t\n\t \n\t \n\t\t<path d=\" M 162.89 226.90 h 0.00 M 163.20 225.10 h 0.00 M 164.84 222.10 h 0.00 M 166.07 220.00 h 0.00 M 166.55 217.90 h 0.00 M 167.59 216.10 h 0.00 M 168.71 214.00 h 0.00 M 170.47 211.90 h 0.00 M 171.80 210.10 h 0.00 M 173.13 208.00 h 0.00 M 174.44 206.20 h 0.00 M 175.26 204.70 h 0.00 M 176.47 202.90 h 0.00 M 176.25 202.30 h 0.00 M 177.31 201.10 h 0.00 M 180.54 197.80 h 0.00 M 184.04 195.10 h 0.00 M 187.18 192.40 h 0.00 M 188.66 190.90 h 0.00 M 188.77 189.70 h 0.00 M 192.04 187.60 h 0.00 M 193.73 186.10 h 0.00 M 193.81 184.60 h 0.00 M 195.97 183.40 h 0.00 M 199.68 181.60 h 0.00 M 204.41 179.20 h 0.00 M 209.54 173.50 h 0.00 M 211.88 172.90 h 0.00 M 217.14 170.80 h 0.00 M 221.09 168.70 h 0.00 M 216.39 170.80 h 0.00 M 227.23 169.90 h 0.00 M 160.47 247.30 h 0.00 M 161.64 248.50 h 0.00 M 162.92 244.00 h 0.00 M 164.38 239.80 h 0.00 M 166.00 236.80 h 0.00 M 167.19 235.30 h 0.00 M 168.95 232.30 h 0.00 M 170.51 230.80 h 0.00 M 170.34 229.30 h 0.00 M 171.25 228.10 h 0.00 M 171.41 226.90 h 0.00 M 172.42 224.80 h 0.00 M 173.04 223.00 h 0.00 M 173.35 220.60 h 0.00 M 174.41 216.40 h 0.00 M 174.72 213.40 h 0.00 M 176.93 210.70 h 0.00 M 179.83 208.60 h 0.00 M 181.07 206.80 h 0.00 M 182.58 205.30 h 0.00 M 184.55 203.20 h 0.00 M 187.37 200.50 h 0.00 M 189.46 199.30 h 0.00 M 191.58 198.10 h 0.00 M 191.33 195.70 h 0.00 M 192.29 194.80 h 0.00 M 192.66 192.70 h 0.00 M 195.38 191.50 h 0.00 M 197.20 189.70 h 0.00 M 199.84 187.60 h 0.00 M 223.07 183.70 h 0.00 M 241.10 185.80 h 0.00 M 151.83 306.40 h 0.00 M 151.96 308.50 h 0.00 M 151.48 308.50 h 0.00 M 151.58 306.40 h 0.00 M 151.58 306.40 h 0.00 M 151.41 305.80 h 0.00 M 151.49 306.40 h 0.00 M 151.34 307.90 h 0.00 M 151.39 308.50 h 0.00 M 151.36 309.70 h 0.00 M 151.52 304.60 h 0.00 M 151.27 300.70 h 0.00 M 151.77 295.90 h 0.00 M 151.98 292.90 h 0.00 M 152.16 290.80 h 0.00 M 152.31 288.40 h 0.00 M 152.49 285.40 h 0.00 M 152.79 280.90 h 0.00 M 152.98 277.30 h 0.00 M 153.02 274.00 h 0.00 M 153.22 271.90 h 0.00 M 153.45 270.10 h 0.00 M 154.19 265.90 h 0.00 M 155.48 262.30 h 0.00 M 155.61 261.70 h 0.00 M 154.89 264.10 h 0.00 M 155.17 262.90 h 0.00 M 154.99 260.20 h 0.00 M 155.20 259.00 h 0.00 M 155.49 256.00 h 0.00 M 155.30 257.50 h 0.00 M 155.71 256.90 h 0.00 M 159.41 265.90 h 0.00 M 160.36 264.10 h 0.00 M 160.18 262.90 h 0.00 M 160.18 261.10 h 0.00 M 161.45 257.50 h 0.00 M 160.84 256.60 h 0.00 M 162.35 253.00 h 0.00 M 162.17 251.80 h 0.00 M 163.08 249.70 h 0.00 M 163.32 246.70 h 0.00 M 163.82 243.70 h 0.00 M 165.12 240.40 h 0.00 M 165.76 237.40 h 0.00 M 166.93 234.40 h 0.00 M 168.01 232.00 h 0.00 M 168.96 229.90 h 0.00 M 170.90 228.10 h 0.00 M 171.96 225.70 h 0.00 M 173.41 223.60 h 0.00 M 174.16 221.80 h 0.00 M 175.01 219.10 h 0.00 M 176.64 215.80 h 0.00 M 176.66 212.80 h 0.00 M 177.43 209.50 h 0.00 M 176.16 206.80 h 0.00 M 178.14 203.80 h 0.00 M 178.23 201.70 h 0.00 M 179.10 199.00 h 0.00 M 180.41 197.20 h 0.00 M 181.56 195.40 h 0.00 M 179.28 196.30 h 0.00 M 182.80 195.10 h 0.00 M 468.38 181.60 h 0.00 M 464.07 178.30 h 0.00 M 465.06 177.40 h 0.00 M 460.85 175.60 h 0.00 M 474.02 173.50 h 0.00 M 487.77 171.40 h 0.00 M 494.29 169.90 h 0.00 M 505.02 168.10 h 0.00 M 488.44 166.30 h 0.00 M 483.04 163.90 h 0.00 M 507.14 161.20 h 0.00 M 501.10 160.00 h 0.00 M 497.20 158.80 h 0.00 M 517.11 155.80 h 0.00 M 544.50 154.60 h 0.00 M 547.97 153.40 h 0.00 M 543.75 151.90 h 0.00 M 497.96 150.70 h 0.00 M 457.39 149.80 h 0.00 M 406.14 150.10 h 0.00 M 398.15 149.50 h 0.00 M 417.99 148.00 h 0.00 M 434.12 146.20 h 0.00 M 438.03 144.40 h 0.00 M 449.39 142.30 h 0.00 M 414.51 140.50 h 0.00 M 411.26 139.00 h 0.00 M 420.14 130.90 h 0.00 M 428.09 127.30 h 0.00 M 434.02 124.00 h 0.00 M 421.19 126.40 h 0.00 M 442.11 126.70 h 0.00 M 163.69 226.30 h 0.00 M 164.89 224.50 h 0.00 M 165.87 223.60 h 0.00 M 165.46 223.60 h 0.00 M 165.57 223.90 h 0.00 M 166.06 225.40 h 0.00 M 166.44 228.70 h 0.00 M 167.03 231.40 h 0.00 M 167.37 232.90 h 0.00 M 167.80 235.60 h 0.00 M 168.45 236.20 h 0.00 M 168.72 237.10 h 0.00 M 169.63 23

In [91]:
Matrix {data: "<svg class=\"poloto\" width=\"800\" height=\"500\" viewBox=\"0 0 800 500\" xmlns=\"http://www.w3.org/2000/svg\">
	<style>
.poloto{
  stroke-linecap:round;
  stroke-linejoin:round;
  font-family:Roboto,sans-serif;
  font-size:16px;
}
.poloto_background{fill:AliceBlue;}
.poloto_scatter{stroke-width:7}
.poloto_line{stroke-width:2}
.poloto_text{fill: black;}
.poloto_name{font-size:24px;dominant-baseline:auto;text-anchor:middle;}
.poloto_where{dominant-baseline:middle;text-anchor:start}
.poloto_text.poloto_legend{font-size:20px;dominant-baseline:middle;text-anchor:start;}
.poloto_text.poloto_ticks.poloto_y{dominant-baseline:middle;text-anchor:end}
.poloto_text.poloto_ticks.poloto_x{dominant-baseline:auto;text-anchor:middle}
.poloto_imgs.poloto_ticks{stroke: black;stroke-width:3;fill:none;stroke-dasharray:none}
.poloto_grid{stroke:gray;stroke-width:0.5}

.poloto0.poloto_stroke{stroke:blue;}
.poloto1.poloto_stroke{stroke:red;}
.poloto2.poloto_stroke{stroke:green;}
.poloto3.poloto_stroke{stroke:gold;}
.poloto4.poloto_stroke{stroke:aqua;}
.poloto5.poloto_stroke{stroke:lime;}
.poloto6.poloto_stroke{stroke:orange;}
.poloto7.poloto_stroke{stroke:chocolate;}
.poloto0.poloto_fill{fill:blue;}
.poloto1.poloto_fill{fill:red;}
.poloto2.poloto_fill{fill:green;}
.poloto3.poloto_fill{fill:gold;}
.poloto4.poloto_fill{fill:aqua;}
.poloto5.poloto_fill{fill:lime;}
.poloto6.poloto_fill{fill:orange;}
.poloto7.poloto_fill{fill:chocolate;}
	</style>
	<circle r=\"1e5\" class=\"poloto_background\" fill=\"white\"/>
	<g id=\"poloto_plot0\" class=\"poloto_plot poloto_imgs poloto_scatter poloto0 poloto_stroke\">
		<path d=\" M 177.93 181.30 h 0.00 M 176.24 183.10 h 0.00 M 174.02 185.50 h 0.00 M 170.96 189.10 h 0.00 M 166.36 193.30 h 0.00 M 164.68 194.80 h 0.00 M 163.49 194.50 h 0.00 M 163.43 193.30 h 0.00 M 163.44 191.20 h 0.00 M 163.76 190.60 h 0.00 M 165.15 190.00 h 0.00 M 167.17 185.50 h 0.00 M 168.63 183.40 h 0.00 M 171.11 180.40 h 0.00 M 174.59 178.00 h 0.00 M 176.40 177.10 h 0.00 M 179.60 174.70 h 0.00 M 183.23 172.90 h 0.00 M 184.86 171.70 h 0.00 M 179.76 172.60 h 0.00 M 181.53 170.80 h 0.00 M 184.15 169.30 h 0.00 M 185.90 168.10 h 0.00 M 191.25 168.10 h 0.00 M 189.85 168.10 h 0.00 M 187.65 167.80 h 0.00 M 191.39 166.30 h 0.00 M 194.11 165.40 h 0.00 M 197.05 165.10 h 0.00 M 199.78 164.20 h 0.00 M 198.84 167.50 h 0.00 M 203.36 168.10 h 0.00 M 151.85 256.30 h 0.00 M 152.10 256.60 h 0.00 M 152.35 257.20 h 0.00 M 152.50 258.40 h 0.00 M 152.74 257.50 h 0.00 M 152.90 258.40 h 0.00 M 153.16 259.90 h 0.00 M 153.35 261.40 h 0.00 M 153.44 262.00 h 0.00 M 153.50 264.40 h 0.00 M 153.78 264.40 h 0.00 M 154.06 265.60 h 0.00 M 154.19 266.20 h 0.00 M 154.55 265.30 h 0.00 M 154.82 265.60 h 0.00 M 155.24 265.60 h 0.00 M 155.72 266.20 h 0.00 M 156.17 264.40 h 0.00 M 156.67 262.90 h 0.00 M 156.57 262.90 h 0.00 M 157.01 259.90 h 0.00 M 157.50 257.50 h 0.00 M 157.54 254.20 h 0.00 M 158.47 253.00 h 0.00 M 159.34 251.20 h 0.00 M 160.31 249.10 h 0.00 M 159.95 245.80 h 0.00 M 158.72 244.60 h 0.00 M 158.68 243.40 h 0.00 M 158.66 242.80 h 0.00 M 158.11 243.70 h 0.00 M 158.56 245.80 h 0.00 M 223.08 151.30 h 0.00 M 228.92 149.50 h 0.00 M 231.68 147.40 h 0.00 M 232.27 144.40 h 0.00 M 235.78 142.60 h 0.00 M 238.71 141.10 h 0.00 M 240.66 139.60 h 0.00 M 242.54 137.80 h 0.00 M 245.49 136.30 h 0.00 M 249.09 134.80 h 0.00 M 252.57 133.30 h 0.00 M 257.03 131.20 h 0.00 M 260.09 129.70 h 0.00 M 262.88 128.50 h 0.00 M 268.31 125.80 h 0.00 M 270.06 125.80 h 0.00 M 279.07 124.90 h 0.00 M 287.20 123.70 h 0.00 M 292.95 123.10 h 0.00 M 289.19 123.10 h 0.00 M 297.33 122.20 h 0.00 M 311.01 120.70 h 0.00 M 314.13 120.10 h 0.00 M 320.35 119.80 h 0.00 M 328.04 118.90 h 0.00 M 330.32 118.60 h 0.00 M 341.64 117.70 h 0.00 M 351.13 116.80 h 0.00 M 359.22 116.50 h 0.00 M 367.61 115.60 h 0.00 M 364.13 116.80 h 0.00 M 372.95 117.40 h 0.00 M 263.03 164.20 h 0.00 M 275.39 161.20 h 0.00 M 278.88 160.00 h 0.00 M 285.70 157.00 h 0.00 M 292.00 154.30 h 0.00 M 295.03 152.80 h 0.00 M 300.26 150.40 h 0.00 M 310.80 148.30 h 0.00 M 317.59 146.20 h 0.00 M 338.84 141.70 h 0.00 M 358.97 140.80 h 0.00 M 362.81 139.30 h 0.00 M 372.10 137.80 h 0.00 M 378.29 138.10 h 0.00 M 395.41 135.40 h 0.00 M 411.01 134.80 h 0.00 M 449.44 134.50 h 0.00 M 473.24 133.90 h 0.00 M 495.53 130.00 h 0.00 M 479.93 128.50 h 0.00 M 493.20 126.40 h 0.00 M 508.89 126.10 h 0.00 M 517.12 125.20 h 0.00 M 533.46 123.70 h 0.00 M 550.12 122.20 h 0.00 M 559.88 125.50 h 0.00 M 580.86 123.10 h 0.00 M 587.04 124.30 h 0.00 M 592.80 123.40 h 0.00 M 604.39 121.90 h 0.00 M 606.18 122.80 h 0.00 M 650.00 121.00 h 0.00 M 169.04 189.10 h 0.00 M 167.19 193.30 h 0.00 M 166.12 196.30 h 0.00 M 166.82 196.00 h 0.00 M 167.95 195.40 h 0.00 M 169.60 193.60 h 0.00 M 170.81 192.10 h 0.00 M 170.12 191.20 h 0.00 M 170.04 190.00 h 0.00 M 170.24 187.60 h 0.00 M 171.20 185.50 h 0.00 M 173.76 183.40 h 0.00 M 176.21 181.30 h 0.00 M 177.72 180.40 h 0.00 M 183.17 177.40 h 0.00 M 185.50 173.20 h 0.00 M 192.94 169.00 h 0.00 M 201.12 164.20 h 0.00 M 212.86 158.20 h 0.00 M 212.29 157.30 h 0.00 M 215.05 157.90 h 0.00 M 220.56 157.60 h 0.00 M 224.39 158.50 h 0.00 M 223.89 157.00 h 0.00 M 227.53 156.70 h 0.00 M 231.31 156.10 h 0.00 M 240.00 155.20 h 0.00 M 251.58 153.10 h 0.00 M 261.58 151.90 h 0.00 M 274.91 150.40 h 0.00 M 275.93 152.80 h 0.00 M 287.14 153.70 h 0.00 M 175.49 222.70 h 0.00 M 175.42 221.80 h 0.00 M 175.88 220.30 h 0.00 M 175.34 219.70 h 0.00 M 175.10 219.40 h 0.00 M 176.14 218.20 h 0.00 M 177.27 214.90 h 0.00 M 177.59 212.80 h 0.00 M 178.93 210.10 h 0.00 M 179.90 207.70 h 0.00 M 181.34 205.30 h 0.00 M 182.60 203.50 h 0.00 M 184.56 200.80 h 0.00 M 187.35 199.00 h 0.00 M 189.51 196.60 h 0.00 M 192.65 194.50 h 0.00 M 194.11 193.00 h 0.00 M 196.13 190.90 h 0.00 M 197.36 188.50 h 0.00 M 197.57 185.80 h 0.00 M 198.99 183.70 h 0.00 M 200.52 181.90 h 0.00 M 199.77 181.30 h 0.00 M 198.85 180.70 h 0.00 M 198.67 179.50 h 0.00 M 194.91 178.00 h 0.00 M 193.44 177.10 h 0.00 M 193.91 176.80 h 0.00 M 194.67 176.50 h 0.00 M 195.10 175.60 h 0.00 M 192.55 179.20 h 0.00 M 195.29 176.50 h 0.00 M 174.28 189.70 h 0.00 M 175.90 188.50 h 0.00 M 178.53 187.60 h 0.00 M 179.86 186.70 h 0.00 M 182.63 185.20 h 0.00 M 182.65 184.90 h 0.00 M 184.97 183.10 h 0.00 M 188.50 179.50 h 0.00 M 190.53 176.80 h 0.00 M 190.27 175.90 h 0.00 M 190.23 174.10 h 0.00 M 189.43 172.00 h 0.00 M 186.81 170.80 h 0.00 M 187.83 169.00 h 0.00 M 190.83 169.00 h 0.00 M 195.31 169.60 h 0.00 M 198.63 169.60 h 0.00 M 203.21 168.70 h 0.00 M 208.09 167.80 h 0.00 M 210.85 166.00 h 0.00 M 216.31 163.90 h 0.00 M 221.07 161.80 h 0.00 M 222.42 160.60 h 0.00 M 226.82 158.50 h 0.00 M 231.39 157.60 h 0.00 M 232.55 156.70 h 0.00 M 235.10 155.50 h 0.00 M 238.02 154.30 h 0.00 M 240.17 154.30 h 0.00 M 242.46 153.70 h 0.00 M 237.73 153.70 h 0.00 M 246.70 157.30 h 0.00 M 199.78 185.20 h 0.00 M 200.53 184.30 h 0.00 M 205.16 182.20 h 0.00 M 205.51 170.50 h 0.00 M 208.88 169.00 h 0.00 M 214.06 167.20 h 0.00 M 214.91 165.70 h 0.00 M 216.67 164.50 h 0.00 M 220.39 161.80 h 0.00 M 223.84 161.80 h 0.00 M 230.04 160.90 h 0.00 M 236.04 158.80 h 0.00 M 238.67 155.20 h 0.00 M 241.43 152.50 h 0.00 M 247.77 151.00 h 0.00 M 254.70 149.80 h 0.00 M 263.27 147.70 h 0.00 M 274.22 145.60 h 0.00 M 281.63 142.90 h 0.00 M 278.03 141.40 h 0.00 M 276.60 142.90 h 0.00 M 276.20 141.70 h 0.00 M 270.55 142.00 h 0.00 M 264.94 141.70 h 0.00 M 263.87 140.50 h 0.00 M 270.55 138.70 h 0.00 M 285.62 136.90 h 0.00 M 295.29 133.90 h 0.00 M 303.90 132.40 h 0.00 M 316.71 130.90 h 0.00 M 306.76 131.80 h 0.00 M 318.11 131.20 h 0.00 M 170.27 221.50 h 0.00 M 171.17 220.00 h 0.00 M 171.47 218.50 h 0.00 M 172.54 216.40 h 0.00 M 173.71 214.90 h 0.00 M 175.32 213.10 h 0.00 M 175.60 210.40 h 0.00 M 176.59 208.60 h 0.00 M 176.32 206.80 h 0.00 M 175.77 205.30 h 0.00 M 175.23 205.30 h 0.00 M 175.15 203.50 h 0.00 M 175.17 201.40 h 0.00 M 176.50 203.50 h 0.00 M 178.05 201.40 h 0.00 M 179.26 200.80 h 0.00 M 181.36 200.80 h 0.00 M 183.69 199.30 h 0.00 M 186.29 196.30 h 0.00 M 186.06 196.90 h 0.00 M 190.20 194.50 h 0.00 M 192.29 191.20 h 0.00 M 190.30 189.40 h 0.00 M 194.61 185.50 h 0.00 M 196.21 185.20 h 0.00 M 196.30 183.10 h 0.00 M 198.54 183.70 h 0.00 M 200.75 182.80 h 0.00 M 202.95 181.90 h 0.00 M 202.97 180.40 h 0.00 M 202.50 181.00 h 0.00 M 206.42 184.90 h 0.00 M 168.99 199.30 h 0.00 M 171.15 198.40 h 0.00 M 173.02 196.60 h 0.00 M 174.40 194.50 h 0.00 M 175.15 193.90 h 0.00 M 175.62 193.00 h 0.00 M 176.65 190.90 h 0.00 M 178.35 189.10 h 0.00 M 180.23 187.00 h 0.00 M 181.27 185.50 h 0.00 M 182.24 183.70 h 0.00 M 182.52 182.20 h 0.00 M 183.13 180.10 h 0.00 M 184.58 178.90 h 0.00 M 187.55 177.10 h 0.00 M 190.82 175.30 h 0.00 M 194.97 173.80 h 0.00 M 200.91 171.10 h 0.00 M 206.07 169.90 h 0.00 M 206.11 169.30 h 0.00 M 209.08 168.10 h 0.00 M 216.07 166.30 h 0.00 M 222.99 164.50 h 0.00 M 231.39 163.00 h 0.00 M 239.63 161.80 h 0.00 M 249.19 160.00 h 0.00 M 259.10 158.50 h 0.00 M 269.46 156.70 h 0.00 M 274.79 155.80 h 0.00 M 279.15 154.90 h 0.00 M 255.97 159.70 h 0.00 M 276.80 158.50 h 0.00 M 151.33 298.60 h 0.00 M 151.33 300.10 h 0.00 M 151.15 304.30 h 0.00 M 151.14 304.30 h 0.00 M 151.22 303.70 h 0.00 M 151.37 303.10 h 0.00 M 151.26 304.30 h 0.00 M 151.35 303.40 h 0.00 M 151.43 302.50 h 0.00 M 151.48 301.30 h 0.00 M 151.40 301.30 h 0.00 M 151.51 300.40 h 0.00 M 151.59 300.10 h 0.00 M 151.43 300.10 h 0.00 M 151.60 298.90 h 0.00 M 151.64 297.40 h 0.00 M 151.80 296.50 h 0.00 M 151.96 294.70 h 0.00 M 152.02 292.60 h 0.00 M 152.23 290.80 h 0.00 M 152.36 288.40 h 0.00 M 152.52 285.70 h 0.00 M 152.82 283.60 h 0.00 M 151.52 289.60 h 0.00 M 151.47 289.00 h 0.00 M 151.71 284.80 h 0.00 M 151.92 282.70 h 0.00 M 152.12 280.60 h 0.00 M 152.25 278.50 h 0.00 M 152.34 276.70 h 0.00 M 152.33 277.90 h 0.00 M 152.44 278.80 h 0.00 M 201.28 178.90 h 0.00 M 203.78 177.10 h 0.00 M 206.51 174.70 h 0.00 M 209.16 172.00 h 0.00 M 213.74 168.70 h 0.00 M 221.10 166.90 h 0.00 M 226.95 164.50 h 0.00 M 235.17 161.80 h 0.00 M 244.53 151.00 h 0.00 M 251.95 148.90 h 0.00 M 264.03 145.90 h 0.00 M 273.03 143.50 h 0.00 M 283.12 140.80 h 0.00 M 287.17 138.10 h 0.00 M 296.50 135.70 h 0.00 M 303.10 133.60 h 0.00 M 317.44 132.70 h 0.00 M 327.16 131.20 h 0.00 M 317.21 130.30 h 0.00 M 306.93 131.50 h 0.00 M 313.55 128.80 h 0.00 M 320.70 129.40 h 0.00 M 325.67 129.10 h 0.00 M 331.21 127.60 h 0.00 M 343.76 125.80 h 0.00 M 412.11 122.50 h 0.00 M 421.49 121.30 h 0.00 M 445.99 119.80 h 0.00 M 471.67 118.90 h 0.00 M 490.75 117.40 h 0.00 M 506.86 117.10 h 0.00 M 554.97 116.50 h 0.00 M 179.50 177.10 h 0.00 M 178.86 177.70 h 0.00 M 175.06 181.60 h 0.00 M 173.38 187.00 h 0.00 M 170.77 190.60 h 0.00 M 170.30 190.60 h 0.00 M 169.94 187.60 h 0.00 M 170.63 185.20 h 0.00 M 169.74 184.30 h 0.00 M 171.45 183.70 h 0.00 M 174.92 180.40 h 0.00 M 176.97 178.00 h 0.00 M 179.54 176.20 h 0.00 M 184.18 173.80 h 0.00 M 187.88 171.70 h 0.00 M 193.96 170.80 h 0.00 M 205.74 167.50 h 0.00 M 212.35 164.20 h 0.00 M 225.74 162.70 h 0.00 M 222.80 163.30 h 0.00 M 226.99 161.20 h 0.00 M 235.78 157.60 h 0.00 M 241.52 156.70 h 0.00 M 248.26 154.90 h 0.00 M 247.07 154.60 h 0.00 M 240.69 152.80 h 0.00 M 240.85 151.60 h 0.00 M 247.70 150.10 h 0.00 M 258.73 147.70 h 0.00 M 263.48 146.50 h 0.00 M 263.27 151.00 h 0.00 M 278.63 153.40 h 0.00 M 155.43 257.80 h 0.00 M 155.53 257.80 h 0.00 M 155.43 258.10 h 0.00 M 155.41 258.70 h 0.00 M 155.53 258.10 h 0.00 M 155.77 257.50 h 0.00 M 155.99 256.90 h 0.00 M 155.96 256.90 h 0.00 M 156.08 256.30 h 0.00 M 156.14 256.00 h 0.00 M 156.13 255.70 h 0.00 M 156.35 254.50 h 0.00 M 156.28 254.50 h 0.00 M 156.42 251.50 h 0.00 M 156.77 247.60 h 0.00 M 157.24 245.20 h 0.00 M 157.79 242.50 h 0.00 M 158.37 240.10 h 0.00 M 158.29 238.30 h 0.00 M 158.38 238.30 h 0.00 M 158.97 236.50 h 0.00 M 159.43 235.00 h 0.00 M 159.85 234.40 h 0.00 M 160.91 233.80 h 0.00 M 161.95 232.60 h 0.00 M 163.26 231.70 h 0.00 M 164.26 229.30 h 0.00 M 165.35 228.40 h 0.00 M 166.34 226.90 h 0.00 M 167.19 225.70 h 0.00 M 167.01 226.60 h 0.00 M 168.87 227.50 h 0.00 M 151.70 335.20 h 0.00 M 151.70 334.60 h 0.00 M 151.73 333.40 h 0.00 M 151.72 331.90 h 0.00 M 151.74 330.40 h 0.00 M 151.77 328.60 h 0.00 M 151.73 327.10 h 0.00 M 151.73 325.60 h 0.00 M 151.94 323.20 h 0.00 M 151.87 322.30 h 0.00 M 151.80 321.40 h 0.00 M 151.97 319.60 h 0.00 M 152.06 317.80 h 0.00 M 152.08 316.60 h 0.00 M 152.06 313.30 h 0.00 M 152.28 310.60 h 0.00 M 152.46 308.20 h 0.00 M 152.53 306.70 h 0.00 M 152.74 304.00 h 0.00 M 152.70 301.90 h 0.00 M 152.92 298.60 h 0.00 M 152.95 296.20 h 0.00 M 153.29 293.80 h 0.00 M 153.18 291.40 h 0.00 M 153.24 289.00 h 0.00 M 153.25 287.20 h 0.00 M 153.28 285.10 h 0.00 M 153.32 283.00 h 0.00 M 153.57 280.30 h 0.00 M 153.75 278.20 h 0.00 M 153.80 279.70 h 0.00 M 153.91 280.00 h 0.00 M 156.59 264.40 h 0.00 M 156.32 265.30 h 0.00 M 156.05 266.20 h 0.00 M 155.44 268.30 h 0.00 M 155.52 269.20 h 0.00 M 155.66 269.50 h 0.00 M 155.88 269.80 h 0.00 M 156.13 269.50 h 0.00 M 156.35 269.20 h 0.00 M 156.59 268.90 h 0.00 M 156.85 267.40 h 0.00 M 157.16 262.00 h 0.00 M 157.44 261.70 h 0.00 M 157.83 259.60 h 0.00 M 158.45 257.80 h 0.00 M 158.70 257.50 h 0.00 M 159.10 256.90 h 0.00 M 159.53 253.30 h 0.00 M 159.73 250.90 h 0.00 M 159.77 248.20 h 0.00 M 159.90 246.10 h 0.00 M 160.18 243.70 h 0.00 M 160.43 240.10 h 0.00 M 161.09 237.40 h 0.00 M 161.75 234.70 h 0.00 M 162.02 232.00 h 0.00 M 162.48 230.80 h 0.00 M 162.98 228.70 h 0.00 M 163.48 226.90 h 0.00 M 163.84 225.10 h 0.00 M 163.67 226.60 h 0.00 M 164.48 227.20 h 0.00 M 257.37 144.70 h 0.00 M 258.63 144.10 h 0.00 M 259.85 143.20 h 0.00 M 261.31 141.70 h 0.00 M 264.24 141.10 h 0.00 M 266.44 140.50 h 0.00 M 268.97 139.00 h 0.00 M 273.64 137.50 h 0.00 M 277.93 136.00 h 0.00 M 281.29 134.80 h 0.00 M 288.74 133.90 h 0.00 M 293.01 133.00 h 0.00 M 296.39 133.00 h 0.00 M 296.51 131.50 h 0.00 M 301.99 129.70 h 0.00 M 307.13 128.50 h 0.00 M 325.23 125.80 h 0.00 M 342.95 124.00 h 0.00 M 353.94 122.20 h 0.00 M 350.76 119.20 h 0.00 M 356.34 117.40 h 0.00 M 367.98 117.10 h 0.00 M 375.40 116.50 h 0.00 M 383.84 115.60 h 0.00 M 390.55 114.40 h 0.00 M 397.59 113.80 h 0.00 M 405.54 113.20 h 0.00 M 408.75 112.90 h 0.00 M 418.26 112.30 h 0.00 M 425.80 111.40 h 0.00 M 418.72 113.20 h 0.00 M 438.41 111.40 h 0.00 M 153.57 292.30 h 0.00 M 153.78 290.80 h 0.00 M 153.88 289.30 h 0.00 M 154.06 287.50 h 0.00 M 154.08 286.00 h 0.00 M 154.39 284.20 h 0.00 M 154.58 282.70 h 0.00 M 154.84 280.90 h 0.00 M 154.96 279.40 h 0.00 M 155.17 277.30 h 0.00 M 155.49 275.20 h 0.00 M 155.78 271.60 h 0.00 M 156.00 269.20 h 0.00 M 156.17 267.10 h 0.00 M 156.46 265.00 h 0.00 M 156.58 262.90 h 0.00 M 156.88 260.80 h 0.00 M 157.34 258.40 h 0.00 M 157.66 256.00 h 0.00 M 157.66 254.20 h 0.00 M 157.69 252.40 h 0.00 M 157.87 250.30 h 0.00 M 158.17 247.30 h 0.00 M 158.79 244.00 h 0.00 M 159.38 242.80 h 0.00 M 159.56 241.30 h 0.00 M 159.97 241.00 h 0.00 M 160.10 241.00 h 0.00 M 160.78 241.00 h 0.00 M 161.45 241.00 h 0.00 M 161.74 242.80 h 0.00 M 162.88 242.50 h 0.00 M 165.82 188.20 h 0.00 M 167.48 185.50 h 0.00 M 170.00 183.10 h 0.00 M 171.88 185.20 h 0.00 M 173.58 183.40 h 0.00 M 176.54 181.60 h 0.00 M 179.24 179.50 h 0.00 M 181.61 176.80 h 0.00 M 182.93 175.30 h 0.00 M 182.91 173.50 h 0.00 M 184.99 171.10 h 0.00 M 186.55 169.60 h 0.00 M 187.93 167.50 h 0.00 M 190.15 166.30 h 0.00 M 193.63 163.90 h 0.00 M 197.25 161.50 h 0.00 M 208.65 161.20 h 0.00 M 213.22 159.70 h 0.00 M 211.98 156.10 h 0.00 M 210.33 156.70 h 0.00 M 217.90 156.10 h 0.00 M 226.38 154.90 h 0.00 M 231.16 152.80 h 0.00 M 234.36 149.20 h 0.00 M 235.71 147.70 h 0.00 M 235.38 146.20 h 0.00 M 238.43 145.00 h 0.00 M 242.45 144.10 h 0.00 M 246.32 143.20 h 0.00 M 246.46 141.70 h 0.00 M 243.95 144.40 h 0.00 M 256.89 143.50 h 0.00 M 177.69 194.80 h 0.00 M 176.42 194.80 h 0.00 M 175.28 194.20 h 0.00 M 175.72 193.30 h 0.00 M 176.89 192.70 h 0.00 M 178.42 190.00 h 0.00 M 175.01 186.70 h 0.00 M 170.83 188.20 h 0.00 M 172.46 185.20 h 0.00 M 171.05 184.00 h 0.00 M 173.39 182.50 h 0.00 M 175.44 179.80 h 0.00 M 178.58 178.00 h 0.00 M 180.99 175.30 h 0.00 M 183.98 173.20 h 0.00 M 188.13 171.40 h 0.00 M 192.38 169.90 h 0.00 M 197.77 167.80 h 0.00 M 203.58 165.40 h 0.00 M 202.95 164.50 h 0.00 M 205.90 163.00 h 0.00 M 208.92 161.80 h 0.00 M 211.13 160.60 h 0.00 M 212.35 158.50 h 0.00 M 216.04 157.90 h 0.00 M 218.99 157.30 h 0.00 M 225.40 157.30 h 0.00 M 230.72 157.60 h 0.00 M 236.61 157.30 h 0.00 M 244.82 157.00 h 0.00 M 245.30 159.40 h 0.00 M 256.03 161.50 h 0.00 M 185.68 182.50 h 0.00 M 185.36 181.90 h 0.00 M 183.96 181.90 h 0.00 M 184.97 180.70 h 0.00 M 186.38 179.20 h 0.00 M 187.84 178.30 h 0.00 M 190.05 176.80 h 0.00 M 192.68 175.90 h 0.00 M 194.75 175.60 h 0.00 M 195.45 175.00 h 0.00 M 198.53 173.20 h 0.00 M 198.32 173.20 h 0.00 M 199.28 172.00 h 0.00 M 201.18 170.50 h 0.00 M 203.14 169.00 h 0.00 M 206.84 167.20 h 0.00 M 212.06 165.70 h 0.00 M 214.86 164.20 h 0.00 M 216.26 163.30 h 0.00 M 213.02 162.70 h 0.00 M 212.05 163.60 h 0.00 M 212.73 164.80 h 0.00 M 208.84 163.90 h 0.00 M 208.92 163.90 h 0.00 M 208.26 163.30 h 0.00 M 209.68 162.70 h 0.00 M 211.29 161.80 h 0.00 M 210.64 161.20 h 0.00 M 211.36 160.90 h 0.00 M 212.29 160.30 h 0.00 M 204.47 163.60 h 0.00 M 206.75 163.00 h 0.00 M 216.40 162.70 h 0.00 M 218.79 160.30 h 0.00 M 221.17 158.20 h 0.00 M 222.10 157.30 h 0.00 M 225.14 153.10 h 0.00 M 228.08 151.60 h 0.00 M 230.29 150.10 h 0.00 M 233.66 148.60 h 0.00 M 237.91 147.40 h 0.00 M 241.52 148.00 h 0.00 M 248.33 146.80 h 0.00 M 253.66 145.90 h 0.00 M 257.58 145.60 h 0.00 M 256.24 144.70 h 0.00 M 259.62 142.60 h 0.00 M 265.11 140.20 h 0.00 M 272.61 139.00 h 0.00 M 278.93 138.10 h 0.00 M 282.63 138.10 h 0.00 M 281.04 138.40 h 0.00 M 285.70 136.90 h 0.00 M 291.57 135.70 h 0.00 M 292.48 135.40 h 0.00 M 299.50 133.90 h 0.00 M 301.87 132.40 h 0.00 M 304.56 132.40 h 0.00 M 312.47 131.50 h 0.00 M 318.76 130.60 h 0.00 M 326.23 129.70 h 0.00 M 341.34 128.50 h 0.00 M 332.32 130.60 h 0.00 M 344.62 129.10 h 0.00 M 197.40 177.40 h 0.00 M 193.33 177.40 h 0.00 M 194.06 177.10 h 0.00 M 195.11 174.70 h 0.00 M 197.45 171.40 h 0.00 M 201.73 169.00 h 0.00 M 204.89 165.40 h 0.00 M 205.40 163.90 h 0.00 M 205.99 162.40 h 0.00 M 207.58 161.20 h 0.00 M 210.79 157.60 h 0.00 M 216.07 154.60 h 0.00 M 218.46 152.50 h 0.00 M 223.36 150.70 h 0.00 M 228.67 149.20 h 0.00 M 233.10 146.80 h 0.00 M 240.07 144.40 h 0.00 M 249.07 142.30 h 0.00 M 255.51 141.10 h 0.00 M 254.69 140.50 h 0.00 M 255.15 139.00 h 0.00 M 259.42 138.40 h 0.00 M 260.38 137.80 h 0.00 M 266.34 135.70 h 0.00 M 272.76 133.90 h 0.00 M 278.08 132.70 h 0.00 M 286.46 131.50 h 0.00 M 296.85 130.90 h 0.00 M 305.68 131.80 h 0.00 M 317.38 130.90 h 0.00 M 312.10 132.40 h 0.00 M 322.78 133.30 h 0.00 M 168.29 217.00 h 0.00 M 168.93 214.30 h 0.00 M 169.80 211.60 h 0.00 M 171.00 209.50 h 0.00 M 172.33 208.00 h 0.00 M 173.59 206.50 h 0.00 M 174.09 204.10 h 0.00 M 174.94 202.00 h 0.00 M 174.92 200.50 h 0.00 M 173.76 201.10 h 0.00 M 174.65 200.20 h 0.00 M 175.25 199.90 h 0.00 M 175.91 199.00 h 0.00 M 177.11 196.90 h 0.00 M 178.97 195.10 h 0.00 M 180.97 191.20 h 0.00 M 183.73 190.60 h 0.00 M 186.61 187.00 h 0.00 M 188.13 185.20 h 0.00 M 188.38 183.40 h 0.00 M 190.18 182.20 h 0.00 M 193.48 180.10 h 0.00 M 195.00 179.80 h 0.00 M 197.68 176.20 h 0.00 M 199.94 175.00 h 0.00 M 199.89 173.80 h 0.00 M 202.15 172.30 h 0.00 M 203.54 171.70 h 0.00 M 206.68 171.10 h 0.00 M 209.39 169.60 h 0.00 M 206.46 173.20 h 0.00 M 214.09 174.40 h 0.00 M 221.98 145.90 h 0.00 M 225.69 144.40 h 0.00 M 228.17 144.10 h 0.00 M 230.50 143.50 h 0.00 M 234.20 135.70 h 0.00 M 238.25 136.60 h 0.00 M 242.33 135.40 h 0.00 M 248.05 135.10 h 0.00 M 254.50 134.20 h 0.00 M 260.44 133.60 h 0.00 M 270.37 132.10 h 0.00 M 275.36 131.50 h 0.00 M 280.17 132.40 h 0.00 M 279.04 131.80 h 0.00 M 285.36 130.90 h 0.00 M 292.28 130.30 h 0.00 M 305.18 128.20 h 0.00 M 316.36 126.40 h 0.00 M 325.79 126.40 h 0.00 M 318.69 126.10 h 0.00 M 320.53 124.90 h 0.00 M 326.47 121.90 h 0.00 M 329.03 121.90 h 0.00 M 336.54 121.30 h 0.00 M 336.51 120.70 h 0.00 M 340.53 120.40 h 0.00 M 348.15 120.10 h 0.00 M 358.82 118.90 h 0.00 M 369.25 118.30 h 0.00 M 378.32 117.10 h 0.00 M 376.85 118.30 h 0.00 M 390.37 117.70 h 0.00 M 159.06 220.60 h 0.00 M 159.11 220.30 h 0.00 M 159.15 219.10 h 0.00 M 159.37 218.20 h 0.00 M 159.80 217.30 h 0.00 M 160.27 216.40 h 0.00 M 160.87 214.00 h 0.00 M 161.39 212.50 h 0.00 M 161.17 211.00 h 0.00 M 161.46 210.10 h 0.00 M 162.01 210.10 h 0.00 M 162.43 208.60 h 0.00 M 162.86 206.50 h 0.00 M 163.56 205.00 h 0.00 M 164.62 202.90 h 0.00 M 165.59 202.60 h 0.00 M 166.68 202.30 h 0.00 M 167.99 200.80 h 0.00 M 168.82 199.60 h 0.00 M 168.87 199.90 h 0.00 M 170.20 197.80 h 0.00 M 171.09 196.30 h 0.00 M 172.71 194.50 h 0.00 M 173.91 192.40 h 0.00 M 175.23 191.20 h 0.00 M 176.06 190.60 h 0.00 M 177.93 190.00 h 0.00 M 179.41 188.80 h 0.00 M 181.56 187.00 h 0.00 M 183.60 184.60 h 0.00 M 180.19 187.00 h 0.00 M 182.95 190.30 h 0.00 M 152.28 319.30 h 0.00 M 152.34 317.80 h 0.00 M 152.42 315.40 h 0.00 M 152.57 313.30 h 0.00 M 152.68 311.50 h 0.00 M 152.82 309.10 h 0.00 M 152.96 307.00 h 0.00 M 153.15 303.70 h 0.00 M 153.27 301.00 h 0.00 M 153.40 298.30 h 0.00 M 153.52 296.50 h 0.00 M 153.74 294.40 h 0.00 M 153.98 291.40 h 0.00 M 154.03 289.00 h 0.00 M 154.18 286.90 h 0.00 M 154.40 283.90 h 0.00 M 154.52 281.80 h 0.00 M 154.92 279.40 h 0.00 M 155.14 277.60 h 0.00 M 154.96 277.00 h 0.00 M 155.17 275.50 h 0.00 M 155.49 274.00 h 0.00 M 155.67 272.50 h 0.00 M 155.72 271.30 h 0.00 M 155.68 269.50 h 0.00 M 155.73 268.00 h 0.00 M 156.71 265.00 h 0.00 M 158.00 262.60 h 0.00 M 158.56 261.40 h 0.00 M 159.03 259.90 h 0.00 M 159.39 260.20 h 0.00 M 160.02 260.50 h 0.00 M 152.06 280.30 h 0.00 M 152.29 277.60 h 0.00 M 152.41 275.20 h 0.00 M 152.53 273.70 h 0.00 M 152.81 271.60 h 0.00 M 152.94 270.10 h 0.00 M 153.14 267.40 h 0.00 M 153.34 265.30 h 0.00 M 153.43 263.50 h 0.00 M 153.61 261.70 h 0.00 M 153.93 259.90 h 0.00 M 154.20 259.30 h 0.00 M 154.21 257.20 h 0.00 M 154.45 255.40 h 0.00 M 154.80 253.30 h 0.00 M 155.12 251.50 h 0.00 M 155.47 248.50 h 0.00 M 155.83 247.30 h 0.00 M 156.34 244.60 h 0.00 M 156.69 241.00 h 0.00 M 157.12 237.10 h 0.00 M 157.54 234.10 h 0.00 M 158.34 231.70 h 0.00 M 159.04 229.00 h 0.00 M 159.92 227.20 h 0.00 M 160.09 226.30 h 0.00 M 160.06 224.20 h 0.00 M 162.24 221.80 h 0.00 M 163.43 219.70 h 0.00 M 164.49 216.70 h 0.00 M 164.05 218.80 h 0.00 M 165.12 219.40 h 0.00 M 162.89 226.90 h 0.00 M 163.20 225.10 h 0.00 M 164.84 222.10 h 0.00 M 166.07 220.00 h 0.00 M 166.55 217.90 h 0.00 M 167.59 216.10 h 0.00 M 168.71 214.00 h 0.00 M 170.47 211.90 h 0.00 M 171.80 210.10 h 0.00 M 173.13 208.00 h 0.00 M 174.44 206.20 h 0.00 M 175.26 204.70 h 0.00 M 176.47 202.90 h 0.00 M 176.25 202.30 h 0.00 M 177.31 201.10 h 0.00 M 180.54 197.80 h 0.00 M 184.04 195.10 h 0.00 M 187.18 192.40 h 0.00 M 188.66 190.90 h 0.00 M 188.77 189.70 h 0.00 M 192.04 187.60 h 0.00 M 193.73 186.10 h 0.00 M 193.81 184.60 h 0.00 M 195.97 183.40 h 0.00 M 199.68 181.60 h 0.00 M 204.41 179.20 h 0.00 M 209.54 173.50 h 0.00 M 211.88 172.90 h 0.00 M 217.14 170.80 h 0.00 M 221.09 168.70 h 0.00 M 216.39 170.80 h 0.00 M 227.23 169.90 h 0.00 M 157.07 206.50 h 0.00 M 157.87 204.40 h 0.00 M 158.07 203.20 h 0.00 M 158.61 201.10 h 0.00 M 159.29 198.40 h 0.00 M 160.26 197.20 h 0.00 M 160.65 196.60 h 0.00 M 160.94 196.00 h 0.00 M 161.29 194.50 h 0.00 M 161.85 193.60 h 0.00 M 162.18 194.50 h 0.00 M 162.90 193.60 h 0.00 M 163.72 191.80 h 0.00 M 164.27 189.70 h 0.00 M 164.21 189.40 h 0.00 M 164.62 189.40 h 0.00 M 164.99 189.10 h 0.00 M 165.40 188.80 h 0.00 M 166.47 187.30 h 0.00 M 165.63 187.60 h 0.00 M 165.93 186.10 h 0.00 M 167.43 184.00 h 0.00 M 167.95 183.10 h 0.00 M 168.42 182.50 h 0.00 M 169.27 181.90 h 0.00 M 169.81 181.00 h 0.00 M 171.50 179.50 h 0.00 M 172.76 178.00 h 0.00 M 173.45 177.40 h 0.00 M 174.12 176.80 h 0.00 M 174.50 176.50 h 0.00 M 174.74 176.50 h 0.00 M 167.27 202.30 h 0.00 M 168.64 201.40 h 0.00 M 169.29 201.10 h 0.00 M 171.51 199.90 h 0.00 M 172.08 199.60 h 0.00 M 172.88 199.30 h 0.00 M 173.06 199.30 h 0.00 M 172.99 199.60 h 0.00 M 172.49 200.20 h 0.00 M 172.89 199.90 h 0.00 M 173.50 200.80 h 0.00 M 174.26 198.10 h 0.00 M 175.05 196.90 h 0.00 M 176.40 198.70 h 0.00 M 177.38 193.90 h 0.00 M 178.39 193.00 h 0.00 M 180.03 191.80 h 0.00 M 181.19 190.90 h 0.00 M 181.42 190.00 h 0.00 M 180.08 190.00 h 0.00 M 179.86 188.80 h 0.00 M 180.91 187.30 h 0.00 M 181.04 186.10 h 0.00 M 182.08 185.80 h 0.00 M 182.53 185.80 h 0.00 M 183.57 186.10 h 0.00 M 185.24 186.40 h 0.00 M 186.96 185.50 h 0.00 M 188.56 185.20 h 0.00 M 189.61 184.30 h 0.00 M 185.97 186.10 h 0.00 M 189.31 187.30 h 0.00 M 152.65 280.90 h 0.00 M 152.83 281.80 h 0.00 M 153.07 276.10 h 0.00 M 153.28 274.00 h 0.00 M 153.46 271.90 h 0.00 M 153.70 269.20 h 0.00 M 153.92 267.70 h 0.00 M 154.14 264.10 h 0.00 M 154.37 259.90 h 0.00 M 154.59 255.70 h 0.00 M 154.92 254.50 h 0.00 M 155.24 252.40 h 0.00 M 155.46 250.30 h 0.00 M 155.79 248.50 h 0.00 M 156.22 246.70 h 0.00 M 156.82 244.60 h 0.00 M 157.50 242.80 h 0.00 M 158.26 241.90 h 0.00 M 158.92 241.00 h 0.00 M 159.40 237.70 h 0.00 M 159.99 234.10 h 0.00 M 160.80 231.70 h 0.00 M 162.23 228.40 h 0.00 M 163.15 227.80 h 0.00 M 164.15 225.10 h 0.00 M 164.98 219.40 h 0.00 M 166.29 216.40 h 0.00 M 167.32 213.40 h 0.00 M 168.91 209.50 h 0.00 M 170.63 206.80 h 0.00 M 171.41 203.50 h 0.00 M 173.77 201.70 h 0.00 M 223.52 146.50 h 0.00 M 228.39 145.30 h 0.00 M 230.58 145.00 h 0.00 M 231.84 143.50 h 0.00 M 235.88 141.70 h 0.00 M 239.82 141.10 h 0.00 M 244.17 139.60 h 0.00 M 246.53 138.70 h 0.00 M 246.11 138.70 h 0.00 M 246.98 138.40 h 0.00 M 252.89 136.90 h 0.00 M 255.38 136.00 h 0.00 M 257.98 135.40 h 0.00 M 260.91 134.80 h 0.00 M 266.39 133.60 h 0.00 M 271.49 133.30 h 0.00 M 277.07 132.40 h 0.00 M 282.33 131.50 h 0.00 M 283.32 131.50 h 0.00 M 276.74 131.20 h 0.00 M 283.55 130.60 h 0.00 M 286.90 130.30 h 0.00 M 292.20 128.50 h 0.00 M 299.05 127.00 h 0.00 M 299.63 125.80 h 0.00 M 304.75 124.60 h 0.00 M 303.60 123.70 h 0.00 M 306.83 123.40 h 0.00 M 309.49 123.10 h 0.00 M 309.98 122.80 h 0.00 M 307.74 123.10 h 0.00 M 312.12 122.50 h 0.00 M 163.69 226.30 h 0.00 M 164.89 224.50 h 0.00 M 165.87 223.60 h 0.00 M 165.46 223.60 h 0.00 M 165.57 223.90 h 0.00 M 166.06 225.40 h 0.00 M 166.44 228.70 h 0.00 M 167.03 231.40 h 0.00 M 167.37 232.90 h 0.00 M 167.80 235.60 h 0.00 M 168.45 236.20 h 0.00 M 168.72 237.10 h 0.00 M 169.63 236.50 h 0.00 M 170.63 235.60 h 0.00 M 173.64 234.40 h 0.00 M 174.74 233.50 h 0.00 M 177.09 231.70 h 0.00 M 179.02 229.30 h 0.00 M 180.00 227.20 h 0.00 M 179.84 225.70 h 0.00 M 181.61 224.50 h 0.00 M 183.44 222.40 h 0.00 M 184.96 219.70 h 0.00 M 186.50 216.70 h 0.00 M 189.34 213.70 h 0.00 M 190.00 211.60 h 0.00 M 189.24 210.40 h 0.00 M 188.30 209.50 h 0.00 M 188.97 209.20 h 0.00 M 188.67 208.30 h 0.00 M 185.30 210.10 h 0.00 M 187.63 215.50 h 0.00 M 155.46 289.00 h 0.00 M 156.09 286.00 h 0.00 M 156.98 282.70 h 0.00 M 158.32 278.50 h 0.00 M 158.72 274.30 h 0.00 M 158.25 273.10 h 0.00 M 158.80 271.00 h 0.00 M 158.23 270.40 h 0.00 M 157.67 269.50 h 0.00 M 157.65 266.80 h 0.00 M 157.33 265.90 h 0.00 M 157.22 264.70 h 0.00 M 157.06 263.50 h 0.00 M 157.12 261.70 h 0.00 M 157.29 261.70 h 0.00 M 157.81 260.20 h 0.00 M 158.28 259.30 h 0.00 M 158.97 256.60 h 0.00 M 158.82 254.80 h 0.00 M 159.24 253.00 h 0.00 M 160.08 250.30 h 0.00 M 160.11 248.80 h 0.00 M 160.53 246.10 h 0.00 M 160.86 244.60 h 0.00 M 162.37 240.10 h 0.00 M 163.03 237.70 h 0.00 M 163.59 235.90 h 0.00 M 164.03 234.40 h 0.00 M 164.00 233.80 h 0.00 M 164.60 232.00 h 0.00 M 163.97 232.00 h 0.00 M 164.35 232.60 h 0.00 M 167.82 202.00 h 0.00 M 168.40 201.10 h 0.00 M 170.14 197.80 h 0.00 M 171.62 195.40 h 0.00 M 172.57 194.50 h 0.00 M 173.48 192.70 h 0.00 M 173.69 192.10 h 0.00 M 174.90 190.00 h 0.00 M 176.47 188.50 h 0.00 M 177.44 187.60 h 0.00 M 178.62 187.00 h 0.00 M 179.78 185.50 h 0.00 M 180.78 183.40 h 0.00 M 182.28 181.90 h 0.00 M 184.15 180.40 h 0.00 M 186.15 178.00 h 0.00 M 189.53 175.90 h 0.00 M 193.42 173.20 h 0.00 M 195.77 170.50 h 0.00 M 195.03 169.30 h 0.00 M 197.46 168.40 h 0.00 M 200.01 165.40 h 0.00 M 202.92 164.20 h 0.00 M 205.50 162.40 h 0.00 M 209.52 161.20 h 0.00 M 213.41 160.60 h 0.00 M 220.92 159.10 h 0.00 M 225.76 157.90 h 0.00 M 229.48 156.70 h 0.00 M 235.06 154.30 h 0.00 M 231.57 155.20 h 0.00 M 235.19 157.30 h 0.00 M 199.64 172.30 h 0.00 M 202.33 169.30 h 0.00 M 203.52 169.30 h 0.00 M 203.59 169.30 h 0.00 M 205.59 167.80 h 0.00 M 207.72 166.90 h 0.00 M 210.45 165.40 h 0.00 M 214.28 163.30 h 0.00 M 217.63 160.30 h 0.00 M 219.27 159.70 h 0.00 M 223.29 157.00 h 0.00 M 228.77 154.00 h 0.00 M 235.09 151.00 h 0.00 M 239.97 149.50 h 0.00 M 245.93 147.10 h 0.00 M 246.37 143.50 h 0.00 M 257.69 141.10 h 0.00 M 260.65 142.60 h 0.00 M 266.48 141.10 h 0.00 M 264.59 140.20 h 0.00 M 255.27 139.30 h 0.00 M 246.73 140.50 h 0.00 M 243.83 140.20 h 0.00 M 247.93 138.70 h 0.00 M 250.36 136.30 h 0.00 M 250.88 136.00 h 0.00 M 253.74 136.90 h 0.00 M 257.91 136.00 h 0.00 M 261.77 134.20 h 0.00 M 267.63 133.30 h 0.00 M 257.19 134.20 h 0.00 M 268.16 133.90 h 0.00 M 205.45 158.20 h 0.00 M 204.26 156.70 h 0.00 M 205.70 154.60 h 0.00 M 209.60 150.10 h 0.00 M 214.06 146.20 h 0.00 M 217.04 144.10 h 0.00 M 219.17 142.90 h 0.00 M 221.79 140.50 h 0.00 M 222.53 137.80 h 0.00 M 227.33 136.60 h 0.00 M 230.84 133.90 h 0.00 M 234.75 131.80 h 0.00 M 237.72 130.90 h 0.00 M 240.32 129.10 h 0.00 M 244.57 128.20 h 0.00 M 246.75 126.40 h 0.00 M 254.58 126.10 h 0.00 M 260.68 125.20 h 0.00 M 262.56 125.50 h 0.00 M 265.81 124.30 h 0.00 M 267.89 123.40 h 0.00 M 273.52 123.40 h 0.00 M 274.61 122.80 h 0.00 M 287.01 121.30 h 0.00 M 291.01 121.00 h 0.00 M 291.72 120.70 h 0.00 M 301.07 119.80 h 0.00 M 309.73 119.50 h 0.00 M 310.88 119.20 h 0.00 M 323.62 118.90 h 0.00 M 322.49 119.20 h 0.00 M 333.50 118.90 h 0.00 M 159.42 203.20 h 0.00 M 158.34 205.30 h 0.00 M 154.60 214.60 h 0.00 M 154.40 215.20 h 0.00 M 154.96 214.00 h 0.00 M 155.62 211.90 h 0.00 M 156.19 210.40 h 0.00 M 156.63 207.70 h 0.00 M 157.36 203.80 h 0.00 M 157.86 202.90 h 0.00 M 158.72 201.40 h 0.00 M 160.03 200.20 h 0.00 M 161.80 197.20 h 0.00 M 164.00 194.20 h 0.00 M 166.13 191.80 h 0.00 M 169.26 187.90 h 0.00 M 172.82 183.70 h 0.00 M 177.04 178.60 h 0.00 M 179.77 177.40 h 0.00 M 175.76 177.70 h 0.00 M 176.85 176.20 h 0.00 M 178.95 175.00 h 0.00 M 184.23 173.50 h 0.00 M 186.34 172.00 h 0.00 M 187.46 170.80 h 0.00 M 187.52 170.20 h 0.00 M 190.70 170.50 h 0.00 M 196.58 169.60 h 0.00 M 200.54 168.70 h 0.00 M 205.78 166.60 h 0.00 M 202.66 172.90 h 0.00 M 208.59 172.30 h 0.00 M 176.29 183.10 h 0.00 M 179.36 181.00 h 0.00 M 182.06 179.50 h 0.00 M 185.14 178.30 h 0.00 M 187.55 176.80 h 0.00 M 186.73 176.50 h 0.00 M 189.05 174.70 h 0.00 M 192.52 173.20 h 0.00 M 194.18 171.40 h 0.00 M 192.76 168.10 h 0.00 M 192.90 166.30 h 0.00 M 191.44 164.80 h 0.00 M 186.98 164.50 h 0.00 M 190.73 162.10 h 0.00 M 195.26 160.60 h 0.00 M 200.41 159.40 h 0.00 M 205.68 155.80 h 0.00 M 211.83 154.90 h 0.00 M 214.97 152.50 h 0.00 M 210.83 151.90 h 0.00 M 217.74 149.80 h 0.00 M 222.54 147.70 h 0.00 M 223.75 147.10 h 0.00 M 225.62 146.50 h 0.00 M 223.92 146.20 h 0.00 M 225.52 145.60 h 0.00 M 226.29 145.90 h 0.00 M 238.83 144.70 h 0.00 M 237.67 145.00 h 0.00 M 236.58 144.40 h 0.00 M 228.12 148.00 h 0.00 M 239.43 147.40 h 0.00 M 194.49 217.00 h 0.00 M 197.57 215.50 h 0.00 M 195.87 215.50 h 0.00 M 197.57 214.30 h 0.00 M 199.12 213.10 h 0.00 M 201.36 211.90 h 0.00 M 202.88 211.00 h 0.00 M 205.52 209.80 h 0.00 M 206.68 208.00 h 0.00 M 200.93 209.20 h 0.00 M 199.79 209.50 h 0.00 M 200.68 207.70 h 0.00 M 199.98 206.80 h 0.00 M 200.74 205.60 h 0.00 M 201.04 205.90 h 0.00 M 202.55 204.40 h 0.00 M 201.07 204.70 h 0.00 M 203.97 203.50 h 0.00 M 201.47 203.50 h 0.00 M 200.16 202.00 h 0.00 M 202.58 200.80 h 0.00 M 205.55 199.60 h 0.00 M 204.65 196.30 h 0.00 M 203.93 194.50 h 0.00 M 205.01 191.80 h 0.00 M 202.69 190.30 h 0.00 M 201.31 189.40 h 0.00 M 204.08 188.20 h 0.00 M 204.53 188.20 h 0.00 M 206.42 187.30 h 0.00 M 204.86 187.00 h 0.00 M 207.00 188.20 h 0.00 M 151.48 309.10 h 0.00 M 151.71 307.30 h 0.00 M 151.49 304.30 h 0.00 M 151.77 301.90 h 0.00 M 151.56 302.20 h 0.00 M 152.07 289.90 h 0.00 M 152.30 289.60 h 0.00 M 152.42 289.30 h 0.00 M 152.52 289.30 h 0.00 M 152.60 287.50 h 0.00 M 152.66 287.80 h 0.00 M 152.47 286.00 h 0.00 M 152.49 284.20 h 0.00 M 152.68 282.10 h 0.00 M 152.88 280.00 h 0.00 M 153.02 278.20 h 0.00 M 153.23 275.50 h 0.00 M 153.63 273.10 h 0.00 M 153.94 270.10 h 0.00 M 154.24 266.50 h 0.00 M 154.51 263.20 h 0.00 M 154.74 261.10 h 0.00 M 154.58 259.00 h 0.00 M 154.99 256.60 h 0.00 M 154.83 253.90 h 0.00 M 154.47 252.70 h 0.00 M 154.54 250.60 h 0.00 M 154.46 248.50 h 0.00 M 154.69 247.00 h 0.00 M 154.94 244.30 h 0.00 M 154.91 245.20 h 0.00 M 155.19 246.40 h 0.00 M 359.12 169.00 h 0.00 M 366.48 167.50 h 0.00 M 375.67 165.70 h 0.00 M 375.79 164.20 h 0.00 M 382.01 163.00 h 0.00 M 391.60 161.50 h 0.00 M 397.26 160.60 h 0.00 M 392.25 160.30 h 0.00 M 388.31 159.70 h 0.00 M 393.82 157.90 h 0.00 M 401.20 157.60 h 0.00 M 408.66 157.30 h 0.00 M 417.62 156.10 h 0.00 M 425.64 154.60 h 0.00 M 429.42 153.40 h 0.00 M 434.38 152.50 h 0.00 M 451.11 151.60 h 0.00 M 454.88 151.90 h 0.00 M 450.08 152.20 h 0.00 M 442.28 151.60 h 0.00 M 449.10 151.60 h 0.00 M 462.50 150.40 h 0.00 M 478.84 148.60 h 0.00 M 462.52 148.60 h 0.00 M 453.84 148.90 h 0.00 M 383.52 149.20 h 0.00 M 360.30 149.50 h 0.00 M 378.19 149.80 h 0.00 M 381.66 151.00 h 0.00 M 392.88 151.00 h 0.00 M 396.79 151.00 h 0.00 M 401.71 151.30 h 0.00 M 218.43 149.80 h 0.00 M 221.59 148.60 h 0.00 M 224.46 147.70 h 0.00 M 226.00 146.80 h 0.00 M 231.56 144.70 h 0.00 M 235.40 143.80 h 0.00 M 239.38 141.40 h 0.00 M 243.90 139.30 h 0.00 M 247.28 137.20 h 0.00 M 250.47 134.20 h 0.00 M 258.13 133.30 h 0.00 M 261.10 131.50 h 0.00 M 265.66 131.20 h 0.00 M 266.35 129.70 h 0.00 M 274.42 128.20 h 0.00 M 279.04 126.70 h 0.00 M 291.16 127.00 h 0.00 M 297.56 126.70 h 0.00 M 306.20 125.50 h 0.00 M 302.59 127.30 h 0.00 M 312.76 126.10 h 0.00 M 318.10 122.20 h 0.00 M 319.63 120.70 h 0.00 M 326.96 120.10 h 0.00 M 331.43 120.40 h 0.00 M 335.79 119.20 h 0.00 M 346.96 117.10 h 0.00 M 359.84 116.80 h 0.00 M 367.94 117.40 h 0.00 M 377.19 116.20 h 0.00 M 380.70 115.90 h 0.00 M 396.14 115.60 h 0.00 M 214.04 158.80 h 0.00 M 215.29 157.30 h 0.00 M 216.89 155.20 h 0.00 M 220.07 154.00 h 0.00 M 224.19 151.60 h 0.00 M 227.50 151.60 h 0.00 M 232.65 149.80 h 0.00 M 236.87 147.40 h 0.00 M 239.25 145.60 h 0.00 M 242.32 143.50 h 0.00 M 250.04 141.40 h 0.00 M 255.19 139.60 h 0.00 M 259.76 138.10 h 0.00 M 264.56 136.60 h 0.00 M 271.12 133.30 h 0.00 M 273.69 131.50 h 0.00 M 281.33 131.80 h 0.00 M 284.36 130.90 h 0.00 M 289.02 130.00 h 0.00 M 282.63 128.20 h 0.00 M 288.29 126.40 h 0.00 M 290.89 127.60 h 0.00 M 294.97 127.30 h 0.00 M 301.23 123.40 h 0.00 M 306.22 122.80 h 0.00 M 311.22 122.80 h 0.00 M 317.54 121.90 h 0.00 M 324.58 121.00 h 0.00 M 328.77 121.30 h 0.00 M 336.72 119.50 h 0.00 M 323.70 122.80 h 0.00 M 339.64 121.30 h 0.00 M 226.74 157.00 h 0.00 M 227.91 156.10 h 0.00 M 228.31 154.00 h 0.00 M 228.05 148.00 h 0.00 M 232.32 144.70 h 0.00 M 236.96 142.00 h 0.00 M 240.11 139.60 h 0.00 M 243.43 136.30 h 0.00 M 247.56 131.80 h 0.00 M 253.59 130.30 h 0.00 M 261.79 128.80 h 0.00 M 262.97 127.90 h 0.00 M 266.75 127.00 h 0.00 M 270.14 125.50 h 0.00 M 277.93 130.00 h 0.00 M 279.40 129.40 h 0.00 M 292.67 128.20 h 0.00 M 304.79 127.60 h 0.00 M 309.55 128.50 h 0.00 M 302.39 128.80 h 0.00 M 309.79 127.00 h 0.00 M 318.88 126.70 h 0.00 M 322.02 126.70 h 0.00 M 325.38 120.40 h 0.00 M 328.70 119.50 h 0.00 M 336.01 118.90 h 0.00 M 341.07 118.30 h 0.00 M 346.85 117.70 h 0.00 M 352.84 117.40 h 0.00 M 363.83 115.90 h 0.00 M 362.82 117.40 h 0.00 M 378.01 115.90 h 0.00 M 155.75 247.30 h 0.00 M 156.19 246.40 h 0.00 M 155.51 250.60 h 0.00 M 155.64 252.70 h 0.00 M 156.38 254.20 h 0.00 M 156.45 256.90 h 0.00 M 157.25 257.20 h 0.00 M 157.48 258.40 h 0.00 M 157.68 260.20 h 0.00 M 157.60 262.90 h 0.00 M 157.44 264.40 h 0.00 M 157.70 267.70 h 0.00 M 157.00 264.40 h 0.00 M 155.70 268.30 h 0.00 M 155.44 266.20 h 0.00 M 155.22 265.60 h 0.00 M 155.14 264.40 h 0.00 M 155.01 263.80 h 0.00 M 153.99 263.80 h 0.00 M 154.58 254.20 h 0.00 M 155.79 246.40 h 0.00 M 156.84 239.50 h 0.00 M 157.87 232.90 h 0.00 M 158.53 229.90 h 0.00 M 158.68 227.20 h 0.00 M 158.88 225.40 h 0.00 M 159.30 223.60 h 0.00 M 157.80 221.80 h 0.00 M 158.28 219.40 h 0.00 M 157.67 219.70 h 0.00 M 156.93 220.00 h 0.00 M 157.81 222.10 h 0.00 M 151.24 313.00 h 0.00 M 151.39 312.40 h 0.00 M 151.40 313.30 h 0.00 M 151.38 322.00 h 0.00 M 151.33 313.00 h 0.00 M 151.05 314.20 h 0.00 M 150.92 315.10 h 0.00 M 150.92 313.90 h 0.00 M 151.00 311.80 h 0.00 M 150.96 311.20 h 0.00 M 150.95 310.90 h 0.00 M 150.98 310.30 h 0.00 M 151.05 306.70 h 0.00 M 151.00 304.30 h 0.00 M 151.07 301.00 h 0.00 M 151.08 298.60 h 0.00 M 151.18 292.00 h 0.00 M 151.24 288.40 h 0.00 M 151.30 284.20 h 0.00 M 151.28 281.50 h 0.00 M 151.32 278.50 h 0.00 M 151.38 277.00 h 0.00 M 151.40 275.20 h 0.00 M 151.57 273.70 h 0.00 M 151.69 272.20 h 0.00 M 151.95 271.60 h 0.00 M 151.84 271.00 h 0.00 M 151.78 271.60 h 0.00 M 151.81 271.60 h 0.00 M 151.82 270.70 h 0.00 M 151.78 272.20 h 0.00 M 151.92 272.20 h 0.00 M 168.00 212.20 h 0.00 M 169.30 210.10 h 0.00 M 170.83 208.30 h 0.00 M 172.21 207.10 h 0.00 M 173.35 206.20 h 0.00 M 174.70 204.70 h 0.00 M 176.36 203.80 h 0.00 M 178.06 202.00 h 0.00 M 179.85 199.90 h 0.00 M 180.69 197.80 h 0.00 M 183.74 195.70 h 0.00 M 185.42 193.00 h 0.00 M 186.33 191.80 h 0.00 M 189.04 189.10 h 0.00 M 191.63 187.00 h 0.00 M 193.49 184.30 h 0.00 M 196.87 182.20 h 0.00 M 200.76 179.80 h 0.00 M 204.39 177.70 h 0.00 M 206.45 175.30 h 0.00 M 209.54 173.50 h 0.00 M 213.22 171.40 h 0.00 M 214.42 167.50 h 0.00 M 217.89 165.40 h 0.00 M 220.83 162.10 h 0.00 M 224.14 161.50 h 0.00 M 230.11 159.70 h 0.00 M 233.29 158.50 h 0.00 M 238.73 156.70 h 0.00 M 242.95 154.90 h 0.00 M 230.27 158.80 h 0.00 M 236.80 159.40 h 0.00 M 239.94 138.40 h 0.00 M 241.67 138.10 h 0.00 M 245.77 136.60 h 0.00 M 249.46 136.00 h 0.00 M 254.44 134.80 h 0.00 M 258.23 134.50 h 0.00 M 263.10 133.90 h 0.00 M 268.78 133.00 h 0.00 M 274.10 132.10 h 0.00 M 280.43 133.30 h 0.00 M 287.34 132.70 h 0.00 M 290.40 132.40 h 0.00 M 293.70 132.10 h 0.00 M 299.38 131.50 h 0.00 M 307.90 130.60 h 0.00 M 317.04 130.00 h 0.00 M 325.34 128.80 h 0.00 M 332.00 128.20 h 0.00 M 333.98 128.20 h 0.00 M 328.74 127.60 h 0.00 M 334.29 126.70 h 0.00 M 339.68 126.10 h 0.00 M 346.23 125.20 h 0.00 M 351.97 124.90 h 0.00 M 358.95 124.30 h 0.00 M 365.19 124.00 h 0.00 M 369.40 123.40 h 0.00 M 377.18 122.80 h 0.00 M 388.29 121.90 h 0.00 M 397.04 121.00 h 0.00 M 390.97 124.00 h 0.00 M 416.47 123.70 h 0.00 M 171.08 224.20 h 0.00 M 172.85 223.60 h 0.00 M 173.39 224.20 h 0.00 M 173.73 223.60 h 0.00 M 174.46 226.60 h 0.00 M 176.10 225.10 h 0.00 M 177.50 225.10 h 0.00 M 179.69 224.20 h 0.00 M 179.49 224.20 h 0.00 M 182.19 223.90 h 0.00 M 182.91 224.50 h 0.00 M 183.07 224.20 h 0.00 M 185.03 225.10 h 0.00 M 186.78 223.00 h 0.00 M 188.15 221.50 h 0.00 M 190.48 218.80 h 0.00 M 194.48 214.60 h 0.00 M 197.46 210.40 h 0.00 M 198.97 207.40 h 0.00 M 191.28 205.90 h 0.00 M 195.17 202.00 h 0.00 M 198.36 199.30 h 0.00 M 194.90 197.80 h 0.00 M 197.81 193.90 h 0.00 M 203.55 191.20 h 0.00 M 200.05 189.40 h 0.00 M 207.13 186.10 h 0.00 M 204.76 183.40 h 0.00 M 207.30 185.20 h 0.00 M 208.96 184.90 h 0.00 M 203.42 186.10 h 0.00 M 211.59 192.10 h 0.00 M 174.13 217.00 h 0.00 M 174.80 215.20 h 0.00 M 174.82 213.40 h 0.00 M 176.29 211.00 h 0.00 M 178.04 208.60 h 0.00 M 179.43 206.20 h 0.00 M 180.19 204.10 h 0.00 M 181.32 202.00 h 0.00 M 181.32 200.20 h 0.00 M 181.46 198.70 h 0.00 M 183.19 196.30 h 0.00 M 183.98 194.20 h 0.00 M 185.16 192.40 h 0.00 M 185.85 193.60 h 0.00 M 188.57 192.40 h 0.00 M 190.66 190.60 h 0.00 M 193.17 190.00 h 0.00 M 196.63 188.80 h 0.00 M 199.52 185.50 h 0.00 M 199.30 184.90 h 0.00 M 203.23 183.10 h 0.00 M 206.03 181.60 h 0.00 M 206.05 180.40 h 0.00 M 208.11 175.00 h 0.00 M 208.60 173.80 h 0.00 M 204.90 174.10 h 0.00 M 203.05 173.50 h 0.00 M 204.08 172.30 h 0.00 M 205.96 170.80 h 0.00 M 207.24 170.20 h 0.00 M 205.69 172.60 h 0.00 M 210.87 173.80 h 0.00 M 219.93 166.60 h 0.00 M 223.41 165.10 h 0.00 M 225.69 163.30 h 0.00 M 226.79 161.50 h 0.00 M 230.15 158.80 h 0.00 M 234.25 157.00 h 0.00 M 236.78 155.20 h 0.00 M 239.89 152.80 h 0.00 M 244.25 151.30 h 0.00 M 246.47 149.50 h 0.00 M 252.09 147.70 h 0.00 M 255.75 145.30 h 0.00 M 258.41 144.10 h 0.00 M 260.17 142.60 h 0.00 M 261.53 140.50 h 0.00 M 263.72 139.60 h 0.00 M 272.56 138.10 h 0.00 M 279.02 136.90 h 0.00 M 284.27 136.60 h 0.00 M 280.76 136.60 h 0.00 M 282.87 135.40 h 0.00 M 288.36 134.50 h 0.00 M 287.94 135.10 h 0.00 M 287.28 135.40 h 0.00 M 286.83 135.10 h 0.00 M 289.51 135.40 h 0.00 M 301.05 133.90 h 0.00 M 307.35 133.60 h 0.00 M 312.89 132.10 h 0.00 M 323.42 130.90 h 0.00 M 313.31 133.30 h 0.00 M 326.87 131.50 h 0.00 M 220.12 155.20 h 0.00 M 223.60 154.30 h 0.00 M 226.16 149.50 h 0.00 M 226.93 145.00 h 0.00 M 230.91 142.60 h 0.00 M 234.45 140.50 h 0.00 M 235.58 138.70 h 0.00 M 239.34 137.20 h 0.00 M 241.77 136.60 h 0.00 M 245.85 135.40 h 0.00 M 254.80 133.90 h 0.00 M 258.63 133.00 h 0.00 M 264.30 132.40 h 0.00 M 266.78 131.50 h 0.00 M 271.08 130.00 h 0.00 M 275.33 129.40 h 0.00 M 283.25 128.50 h 0.00 M 289.12 127.60 h 0.00 M 293.26 127.60 h 0.00 M 292.75 127.00 h 0.00 M 300.71 126.40 h 0.00 M 304.92 125.80 h 0.00 M 310.05 125.50 h 0.00 M 315.31 125.20 h 0.00 M 320.11 123.70 h 0.00 M 324.95 122.80 h 0.00 M 334.08 121.90 h 0.00 M 341.11 120.70 h 0.00 M 349.07 120.10 h 0.00 M 361.54 119.20 h 0.00 M 356.84 121.60 h 0.00 M 373.11 118.90 h 0.00 M 151.45 301.30 h 0.00 M 151.59 300.70 h 0.00 M 151.65 302.50 h 0.00 M 151.85 300.40 h 0.00 M 152.00 299.20 h 0.00 M 152.33 297.40 h 0.00 M 152.61 293.80 h 0.00 M 152.75 290.80 h 0.00 M 152.86 288.10 h 0.00 M 153.10 284.50 h 0.00 M 153.20 281.80 h 0.00 M 153.38 278.80 h 0.00 M 153.70 274.90 h 0.00 M 153.95 271.30 h 0.00 M 154.27 266.80 h 0.00 M 154.62 263.80 h 0.00 M 155.24 260.20 h 0.00 M 155.76 257.20 h 0.00 M 156.28 254.80 h 0.00 M 156.60 251.80 h 0.00 M 156.90 249.40 h 0.00 M 157.57 248.20 h 0.00 M 156.65 248.80 h 0.00 M 156.68 247.30 h 0.00 M 157.00 246.40 h 0.00 M 157.19 244.90 h 0.00 M 157.08 244.30 h 0.00 M 157.03 244.00 h 0.00 M 157.45 243.40 h 0.00 M 157.85 242.50 h 0.00 M 157.93 242.80 h 0.00 M 158.36 242.50 h 0.00 M 158.72 211.60 h 0.00 M 158.10 213.40 h 0.00 M 155.47 229.00 h 0.00 M 154.46 237.40 h 0.00 M 153.31 235.90 h 0.00 M 152.77 235.90 h 0.00 M 152.12 240.70 h 0.00 M 152.19 238.00 h 0.00 M 152.38 235.90 h 0.00 M 152.54 234.10 h 0.00 M 152.87 232.00 h 0.00 M 153.25 229.30 h 0.00 M 153.70 226.30 h 0.00 M 154.22 221.80 h 0.00 M 154.81 218.20 h 0.00 M 155.29 216.40 h 0.00 M 155.81 213.40 h 0.00 M 156.41 212.50 h 0.00 M 157.01 208.30 h 0.00 M 157.21 210.10 h 0.00 M 157.68 209.20 h 0.00 M 158.32 209.50 h 0.00 M 159.29 207.10 h 0.00 M 160.34 203.20 h 0.00 M 161.39 203.20 h 0.00 M 160.82 202.90 h 0.00 M 160.70 202.00 h 0.00 M 161.25 200.50 h 0.00 M 162.20 198.70 h 0.00 M 163.12 197.20 h 0.00 M 163.61 200.80 h 0.00 M 165.36 194.50 h 0.00 M 201.06 172.90 h 0.00 M 204.06 170.80 h 0.00 M 205.56 168.40 h 0.00 M 206.01 165.70 h 0.00 M 208.33 163.30 h 0.00 M 210.98 161.20 h 0.00 M 213.61 159.40 h 0.00 M 217.17 157.60 h 0.00 M 221.68 156.10 h 0.00 M 224.95 154.60 h 0.00 M 231.17 152.50 h 0.00 M 236.37 151.00 h 0.00 M 241.78 149.80 h 0.00 M 244.35 148.90 h 0.00 M 248.53 146.80 h 0.00 M 254.10 145.00 h 0.00 M 265.96 143.20 h 0.00 M 272.65 142.60 h 0.00 M 275.66 142.00 h 0.00 M 271.20 141.70 h 0.00 M 269.63 139.60 h 0.00 M 270.34 138.40 h 0.00 M 269.78 137.80 h 0.00 M 272.54 136.00 h 0.00 M 276.71 134.80 h 0.00 M 281.97 133.30 h 0.00 M 291.06 131.50 h 0.00 M 299.61 130.90 h 0.00 M 304.06 129.70 h 0.00 M 313.17 127.60 h 0.00 M 293.66 130.30 h 0.00 M 303.85 128.50 h 0.00 M 161.79 226.30 h 0.00 M 160.85 227.50 h 0.00 M 159.81 230.80 h 0.00 M 159.56 234.70 h 0.00 M 159.87 233.50 h 0.00 M 160.66 232.00 h 0.00 M 161.01 230.80 h 0.00 M 161.56 228.40 h 0.00 M 162.00 225.10 h 0.00 M 162.46 222.70 h 0.00 M 162.80 220.60 h 0.00 M 163.40 217.30 h 0.00 M 164.19 214.90 h 0.00 M 165.44 210.70 h 0.00 M 167.52 205.90 h 0.00 M 169.33 202.60 h 0.00 M 171.56 199.30 h 0.00 M 174.32 196.00 h 0.00 M 176.81 192.70 h 0.00 M 176.29 191.50 h 0.00 M 177.99 189.70 h 0.00 M 183.20 187.30 h 0.00 M 188.34 184.60 h 0.00 M 189.63 182.20 h 0.00 M 191.58 181.30 h 0.00 M 189.95 180.40 h 0.00 M 190.19 180.10 h 0.00 M 192.47 180.40 h 0.00 M 195.94 177.10 h 0.00 M 198.40 176.20 h 0.00 M 195.86 176.50 h 0.00 M 197.98 178.30 h 0.00 M 151.83 306.40 h 0.00 M 151.96 308.50 h 0.00 M 151.48 308.50 h 0.00 M 151.58 306.40 h 0.00 M 151.58 306.40 h 0.00 M 151.41 305.80 h 0.00 M 151.49 306.40 h 0.00 M 151.34 307.90 h 0.00 M 151.39 308.50 h 0.00 M 151.36 309.70 h 0.00 M 151.52 304.60 h 0.00 M 151.27 300.70 h 0.00 M 151.77 295.90 h 0.00 M 151.98 292.90 h 0.00 M 152.16 290.80 h 0.00 M 152.31 288.40 h 0.00 M 152.49 285.40 h 0.00 M 152.79 280.90 h 0.00 M 152.98 277.30 h 0.00 M 153.02 274.00 h 0.00 M 153.22 271.90 h 0.00 M 153.45 270.10 h 0.00 M 154.19 265.90 h 0.00 M 155.48 262.30 h 0.00 M 155.61 261.70 h 0.00 M 154.89 264.10 h 0.00 M 155.17 262.90 h 0.00 M 154.99 260.20 h 0.00 M 155.20 259.00 h 0.00 M 155.49 256.00 h 0.00 M 155.30 257.50 h 0.00 M 155.71 256.90 h 0.00 M 154.92 262.00 h 0.00 M 155.29 260.20 h 0.00 M 155.52 259.00 h 0.00 M 155.83 257.50 h 0.00 M 156.04 256.90 h 0.00 M 156.32 255.10 h 0.00 M 156.62 253.90 h 0.00 M 156.89 252.40 h 0.00 M 157.16 250.90 h 0.00 M 157.43 249.40 h 0.00 M 157.72 247.90 h 0.00 M 158.04 248.50 h 0.00 M 158.33 246.40 h 0.00 M 158.75 245.50 h 0.00 M 159.30 243.70 h 0.00 M 159.96 240.10 h 0.00 M 160.71 238.00 h 0.00 M 161.23 235.00 h 0.00 M 162.27 231.70 h 0.00 M 162.65 230.20 h 0.00 M 163.55 227.80 h 0.00 M 165.54 224.50 h 0.00 M 167.50 222.40 h 0.00 M 168.42 220.00 h 0.00 M 169.40 220.00 h 0.00 M 168.05 217.90 h 0.00 M 167.25 216.70 h 0.00 M 167.70 215.20 h 0.00 M 168.91 214.00 h 0.00 M 170.15 210.70 h 0.00 M 170.10 210.40 h 0.00 M 171.77 210.40 h 0.00 M 172.48 185.20 h 0.00 M 171.49 186.10 h 0.00 M 172.51 184.60 h 0.00 M 173.91 180.70 h 0.00 M 175.74 177.70 h 0.00 M 178.20 176.20 h 0.00 M 180.46 173.50 h 0.00 M 182.97 170.20 h 0.00 M 185.07 167.20 h 0.00 M 187.17 165.10 h 0.00 M 189.59 162.10 h 0.00 M 191.29 160.30 h 0.00 M 193.89 158.20 h 0.00 M 195.74 156.70 h 0.00 M 199.80 157.30 h 0.00 M 201.87 155.50 h 0.00 M 206.65 154.00 h 0.00 M 212.95 152.50 h 0.00 M 218.66 150.70 h 0.00 M 222.23 149.50 h 0.00 M 228.51 147.70 h 0.00 M 235.25 145.90 h 0.00 M 238.80 144.40 h 0.00 M 241.58 141.10 h 0.00 M 245.38 140.50 h 0.00 M 251.12 139.60 h 0.00 M 256.23 138.40 h 0.00 M 263.06 137.50 h 0.00 M 270.95 136.90 h 0.00 M 282.18 135.70 h 0.00 M 283.48 137.20 h 0.00 M 294.22 137.20 h 0.00 M 159.10 280.90 h 0.00 M 159.32 279.40 h 0.00 M 159.35 276.70 h 0.00 M 159.77 273.10 h 0.00 M 159.30 269.50 h 0.00 M 160.24 267.70 h 0.00 M 160.88 265.30 h 0.00 M 160.33 265.00 h 0.00 M 160.47 262.30 h 0.00 M 160.74 260.50 h 0.00 M 160.21 260.50 h 0.00 M 160.10 260.20 h 0.00 M 160.18 259.00 h 0.00 M 160.94 257.50 h 0.00 M 161.57 254.20 h 0.00 M 162.77 253.00 h 0.00 M 165.44 250.00 h 0.00 M 165.17 250.30 h 0.00 M 165.02 250.30 h 0.00 M 164.71 247.60 h 0.00 M 164.85 247.00 h 0.00 M 165.36 245.50 h 0.00 M 165.47 243.10 h 0.00 M 166.33 240.70 h 0.00 M 165.66 238.90 h 0.00 M 165.33 236.80 h 0.00 M 167.32 237.70 h 0.00 M 168.96 236.50 h 0.00 M 169.86 233.20 h 0.00 M 170.78 231.10 h 0.00 M 170.30 233.20 h 0.00 M 171.23 233.20 h 0.00 M 158.07 209.20 h 0.00 M 158.69 207.70 h 0.00 M 159.23 205.00 h 0.00 M 160.08 202.60 h 0.00 M 160.83 201.10 h 0.00 M 161.63 200.20 h 0.00 M 162.25 198.40 h 0.00 M 163.25 196.60 h 0.00 M 164.00 195.10 h 0.00 M 164.78 192.70 h 0.00 M 166.01 193.60 h 0.00 M 166.00 193.00 h 0.00 M 166.77 191.80 h 0.00 M 168.02 190.30 h 0.00 M 169.40 195.10 h 0.00 M 171.15 187.00 h 0.00 M 173.35 185.80 h 0.00 M 175.44 184.30 h 0.00 M 177.26 183.70 h 0.00 M 178.17 186.10 h 0.00 M 180.60 178.90 h 0.00 M 183.74 176.80 h 0.00 M 188.24 174.70 h 0.00 M 190.65 173.20 h 0.00 M 192.55 172.00 h 0.00 M 194.63 170.80 h 0.00 M 198.15 169.90 h 0.00 M 200.52 168.70 h 0.00 M 202.58 167.20 h 0.00 M 203.09 166.60 h 0.00 M 200.98 166.00 h 0.00 M 204.63 165.40 h 0.00 M 154.82 288.10 h 0.00 M 155.01 287.80 h 0.00 M 155.06 287.50 h 0.00 M 155.11 287.80 h 0.00 M 155.08 287.80 h 0.00 M 155.38 287.50 h 0.00 M 155.48 287.20 h 0.00 M 155.63 286.90 h 0.00 M 155.95 286.30 h 0.00 M 156.32 285.40 h 0.00 M 156.60 283.60 h 0.00 M 156.92 281.80 h 0.00 M 156.86 280.60 h 0.00 M 157.26 278.50 h 0.00 M 157.66 276.40 h 0.00 M 158.08 274.30 h 0.00 M 158.34 271.90 h 0.00 M 158.61 268.30 h 0.00 M 158.89 264.70 h 0.00 M 158.96 262.30 h 0.00 M 159.15 259.60 h 0.00 M 159.21 255.40 h 0.00 M 159.49 253.00 h 0.00 M 159.51 251.20 h 0.00 M 159.86 249.40 h 0.00 M 160.41 248.50 h 0.00 M 160.83 247.90 h 0.00 M 161.34 247.30 h 0.00 M 162.06 246.40 h 0.00 M 162.54 246.10 h 0.00 M 162.53 246.10 h 0.00 M 163.68 246.70 h 0.00 M 156.60 280.00 h 0.00 M 157.00 278.80 h 0.00 M 157.59 277.30 h 0.00 M 157.72 277.00 h 0.00 M 157.98 275.80 h 0.00 M 158.35 274.90 h 0.00 M 158.68 273.10 h 0.00 M 158.66 272.50 h 0.00 M 158.73 271.00 h 0.00 M 158.95 269.50 h 0.00 M 159.29 267.70 h 0.00 M 159.56 265.60 h 0.00 M 159.73 263.80 h 0.00 M 160.32 261.40 h 0.00 M 161.23 257.50 h 0.00 M 162.16 254.80 h 0.00 M 163.07 252.10 h 0.00 M 163.84 250.30 h 0.00 M 164.05 250.60 h 0.00 M 164.23 249.40 h 0.00 M 164.31 248.50 h 0.00 M 164.73 247.60 h 0.00 M 164.88 246.10 h 0.00 M 165.31 244.60 h 0.00 M 165.96 241.90 h 0.00 M 166.64 239.80 h 0.00 M 166.95 237.70 h 0.00 M 167.56 237.10 h 0.00 M 168.87 236.50 h 0.00 M 169.40 236.20 h 0.00 M 169.05 237.10 h 0.00 M 170.91 236.80 h 0.00 M 154.79 276.40 h 0.00 M 154.84 277.60 h 0.00 M 154.72 277.60 h 0.00 M 155.11 275.80 h 0.00 M 154.56 276.70 h 0.00 M 154.70 276.10 h 0.00 M 155.02 275.80 h 0.00 M 155.20 276.10 h 0.00 M 155.09 276.70 h 0.00 M 155.30 276.10 h 0.00 M 155.51 274.60 h 0.00 M 155.81 272.50 h 0.00 M 156.01 269.80 h 0.00 M 156.42 265.60 h 0.00 M 156.90 262.30 h 0.00 M 157.46 258.40 h 0.00 M 158.10 254.80 h 0.00 M 158.78 251.50 h 0.00 M 159.37 248.20 h 0.00 M 160.00 244.30 h 0.00 M 160.87 241.30 h 0.00 M 161.37 239.80 h 0.00 M 161.71 235.60 h 0.00 M 162.28 233.80 h 0.00 M 162.11 232.90 h 0.00 M 161.69 231.40 h 0.00 M 161.58 230.80 h 0.00 M 161.85 229.60 h 0.00 M 162.28 228.40 h 0.00 M 162.31 227.50 h 0.00 M 161.73 229.00 h 0.00 M 162.53 230.50 h 0.00 M 153.51 269.80 h 0.00 M 153.61 268.90 h 0.00 M 153.88 267.40 h 0.00 M 154.13 266.20 h 0.00 M 154.48 264.40 h 0.00 M 154.91 262.60 h 0.00 M 155.35 260.20 h 0.00 M 155.60 258.70 h 0.00 M 155.96 256.60 h 0.00 M 156.54 254.20 h 0.00 M 156.87 252.70 h 0.00 M 157.29 251.20 h 0.00 M 157.59 249.10 h 0.00 M 158.29 245.20 h 0.00 M 159.13 242.50 h 0.00 M 160.11 239.80 h 0.00 M 161.19 237.10 h 0.00 M 162.29 234.10 h 0.00 M 162.77 232.00 h 0.00 M 163.75 230.50 h 0.00 M 164.98 227.50 h 0.00 M 165.94 223.60 h 0.00 M 167.31 220.60 h 0.00 M 168.03 217.90 h 0.00 M 168.68 214.30 h 0.00 M 169.54 211.30 h 0.00 M 170.93 208.30 h 0.00 M 172.21 206.80 h 0.00 M 174.03 206.50 h 0.00 M 175.20 206.50 h 0.00 M 173.75 207.40 h 0.00 M 176.99 210.10 h 0.00 M 155.00 232.30 h 0.00 M 155.31 231.40 h 0.00 M 155.56 232.60 h 0.00 M 155.96 229.00 h 0.00 M 156.82 226.90 h 0.00 M 156.56 227.50 h 0.00 M 156.21 228.10 h 0.00 M 157.07 226.30 h 0.00 M 158.43 223.30 h 0.00 M 158.41 222.40 h 0.00 M 158.53 220.90 h 0.00 M 158.76 217.00 h 0.00 M 159.88 214.00 h 0.00 M 159.55 218.50 h 0.00 M 159.45 217.00 h 0.00 M 159.15 217.00 h 0.00 M 159.60 216.70 h 0.00 M 160.53 216.10 h 0.00 M 161.39 214.60 h 0.00 M 160.53 215.50 h 0.00 M 160.26 215.20 h 0.00 M 161.11 215.20 h 0.00 M 160.90 212.80 h 0.00 M 161.57 211.60 h 0.00 M 161.82 211.90 h 0.00 M 163.34 207.10 h 0.00 M 164.34 209.20 h 0.00 M 165.06 209.80 h 0.00 M 165.62 207.40 h 0.00 M 168.11 209.50 h 0.00 M 167.34 208.30 h 0.00 M 168.28 207.70 h 0.00 M 152.66 254.80 h 0.00 M 153.08 252.40 h 0.00 M 153.73 248.80 h 0.00 M 154.46 245.50 h 0.00 M 155.24 242.50 h 0.00 M 156.01 239.50 h 0.00 M 156.77 236.50 h 0.00 M 157.55 233.80 h 0.00 M 158.25 230.80 h 0.00 M 159.01 228.10 h 0.00 M 160.03 224.80 h 0.00 M 161.14 222.10 h 0.00 M 162.37 218.80 h 0.00 M 163.92 215.50 h 0.00 M 165.79 212.20 h 0.00 M 168.18 208.60 h 0.00 M 171.18 205.00 h 0.00 M 174.90 201.10 h 0.00 M 177.80 198.40 h 0.00 M 180.56 195.40 h 0.00 M 184.18 192.70 h 0.00 M 188.14 190.00 h 0.00 M 191.47 187.30 h 0.00 M 194.15 184.90 h 0.00 M 196.47 182.50 h 0.00 M 198.06 180.10 h 0.00 M 200.29 178.00 h 0.00 M 203.19 175.90 h 0.00 M 207.97 173.50 h 0.00 M 212.38 171.40 h 0.00 M 214.49 170.80 h 0.00 M 223.16 169.60 h 0.00 M 150.00 328.60 h 0.00 M 150.07 328.00 h 0.00 M 150.00 329.80 h 0.00 M 150.11 328.30 h 0.00 M 150.16 327.40 h 0.00 M 150.15 325.90 h 0.00 M 150.28 322.30 h 0.00 M 150.41 318.40 h 0.00 M 150.54 315.10 h 0.00 M 150.71 311.20 h 0.00 M 150.73 309.10 h 0.00 M 150.95 305.20 h 0.00 M 151.11 302.80 h 0.00 M 151.25 298.90 h 0.00 M 151.43 295.90 h 0.00 M 151.61 292.90 h 0.00 M 151.89 290.80 h 0.00 M 152.12 286.90 h 0.00 M 152.33 283.90 h 0.00 M 152.47 281.50 h 0.00 M 152.64 279.40 h 0.00 M 152.89 277.90 h 0.00 M 152.92 276.10 h 0.00 M 153.10 271.90 h 0.00 M 153.30 270.10 h 0.00 M 153.87 268.00 h 0.00 M 154.14 267.10 h 0.00 M 153.82 266.50 h 0.00 M 153.96 264.70 h 0.00 M 154.02 263.20 h 0.00 M 153.88 264.10 h 0.00 M 154.07 266.20 h 0.00 M 219.26 148.60 h 0.00 M 223.53 146.50 h 0.00 M 227.50 144.70 h 0.00 M 231.16 142.60 h 0.00 M 236.66 138.10 h 0.00 M 241.74 138.70 h 0.00 M 251.13 136.30 h 0.00 M 257.92 134.50 h 0.00 M 256.36 131.50 h 0.00 M 265.41 129.70 h 0.00 M 289.85 126.10 h 0.00 M 293.06 126.70 h 0.00 M 293.92 126.40 h 0.00 M 296.31 124.30 h 0.00 M 311.51 121.90 h 0.00 M 331.68 120.70 h 0.00 M 356.06 118.90 h 0.00 M 362.96 118.60 h 0.00 M 385.43 118.00 h 0.00 M 360.84 119.20 h 0.00 M 370.77 117.70 h 0.00 M 386.90 116.80 h 0.00 M 399.53 116.20 h 0.00 M 405.64 115.30 h 0.00 M 401.66 114.40 h 0.00 M 380.34 114.10 h 0.00 M 374.79 113.50 h 0.00 M 395.02 112.30 h 0.00 M 416.60 111.40 h 0.00 M 415.31 111.70 h 0.00 M 397.08 112.30 h 0.00 M 460.85 111.70 h 0.00 M 159.86 242.50 h 0.00 M 160.20 239.20 h 0.00 M 161.07 235.60 h 0.00 M 161.90 232.60 h 0.00 M 162.64 229.60 h 0.00 M 163.45 226.60 h 0.00 M 163.71 223.90 h 0.00 M 164.32 220.90 h 0.00 M 164.79 218.50 h 0.00 M 165.26 217.30 h 0.00 M 165.72 214.90 h 0.00 M 166.16 214.30 h 0.00 M 166.64 212.20 h 0.00 M 167.22 210.70 h 0.00 M 167.85 209.20 h 0.00 M 168.97 207.70 h 0.00 M 170.50 206.20 h 0.00 M 171.46 204.70 h 0.00 M 172.29 204.10 h 0.00 M 171.86 204.10 h 0.00 M 172.53 202.30 h 0.00 M 173.85 201.70 h 0.00 M 174.15 200.80 h 0.00 M 175.46 199.90 h 0.00 M 176.86 200.20 h 0.00 M 178.30 199.60 h 0.00 M 180.06 198.70 h 0.00 M 181.75 197.80 h 0.00 M 183.30 196.00 h 0.00 M 184.75 195.10 h 0.00 M 182.30 198.40 h 0.00 M 187.56 197.50 h 0.00 M 156.34 253.00 h 0.00 M 156.40 250.00 h 0.00 M 156.43 248.50 h 0.00 M 156.41 247.60 h 0.00 M 156.65 246.10 h 0.00 M 157.11 243.40 h 0.00 M 157.62 240.40 h 0.00 M 157.97 237.40 h 0.00 M 158.26 236.80 h 0.00 M 158.90 232.30 h 0.00 M 159.40 230.20 h 0.00 M 159.80 228.70 h 0.00 M 159.92 227.50 h 0.00 M 160.28 226.60 h 0.00 M 161.06 225.40 h 0.00 M 161.81 223.60 h 0.00 M 162.57 222.10 h 0.00 M 163.45 220.30 h 0.00 M 164.02 218.50 h 0.00 M 163.41 218.20 h 0.00 M 164.01 215.80 h 0.00 M 165.05 213.40 h 0.00 M 165.65 211.30 h 0.00 M 166.32 209.80 h 0.00 M 167.55 207.70 h 0.00 M 168.85 205.90 h 0.00 M 170.32 204.10 h 0.00 M 171.46 202.30 h 0.00 M 170.91 201.40 h 0.00 M 170.36 200.80 h 0.00 M 169.96 203.80 h 0.00 M 172.85 199.90 h 0.00 M 156.58 245.20 h 0.00 M 156.36 244.90 h 0.00 M 156.77 243.70 h 0.00 M 157.24 241.90 h 0.00 M 157.20 241.00 h 0.00 M 157.65 239.20 h 0.00 M 157.73 238.30 h 0.00 M 158.04 236.80 h 0.00 M 158.22 241.90 h 0.00 M 158.04 235.00 h 0.00 M 158.66 233.20 h 0.00 M 158.88 232.30 h 0.00 M 159.14 231.10 h 0.00 M 159.54 229.90 h 0.00 M 160.22 228.40 h 0.00 M 160.98 226.90 h 0.00 M 161.86 225.70 h 0.00 M 162.72 224.20 h 0.00 M 163.26 222.70 h 0.00 M 162.70 221.80 h 0.00 M 163.09 220.90 h 0.00 M 163.64 220.00 h 0.00 M 164.02 220.30 h 0.00 M 164.41 219.10 h 0.00 M 165.56 218.20 h 0.00 M 166.94 216.10 h 0.00 M 168.14 214.60 h 0.00 M 169.67 214.90 h 0.00 M 170.62 214.90 h 0.00 M 171.21 210.40 h 0.00 M 169.15 213.70 h 0.00 M 172.34 213.70 h 0.00 M 154.71 299.20 h 0.00 M 155.24 297.40 h 0.00 M 155.70 300.70 h 0.00 M 156.06 298.00 h 0.00 M 156.11 285.10 h 0.00 M 156.50 288.70 h 0.00 M 156.89 286.30 h 0.00 M 158.28 283.60 h 0.00 M 158.57 287.20 h 0.00 M 158.77 276.10 h 0.00 M 159.33 272.80 h 0.00 M 159.94 270.70 h 0.00 M 160.46 269.20 h 0.00 M 161.10 268.30 h 0.00 M 161.72 267.70 h 0.00 M 162.47 265.60 h 0.00 M 163.35 260.50 h 0.00 M 164.10 259.00 h 0.00 M 164.49 256.60 h 0.00 M 163.69 254.20 h 0.00 M 163.98 254.20 h 0.00 M 165.28 253.90 h 0.00 M 163.84 252.10 h 0.00 M 163.98 250.90 h 0.00 M 166.23 248.80 h 0.00 M 166.14 247.60 h 0.00 M 167.26 246.70 h 0.00 M 166.50 245.80 h 0.00 M 165.89 245.80 h 0.00 M 165.33 245.80 h 0.00 M 164.51 247.00 h 0.00 M 164.48 247.60 h 0.00 M 159.41 265.90 h 0.00 M 160.36 264.10 h 0.00 M 160.18 262.90 h 0.00 M 160.18 261.10 h 0.00 M 161.45 257.50 h 0.00 M 160.84 256.60 h 0.00 M 162.35 253.00 h 0.00 M 162.17 251.80 h 0.00 M 163.08 249.70 h 0.00 M 163.32 246.70 h 0.00 M 163.82 243.70 h 0.00 M 165.12 240.40 h 0.00 M 165.76 237.40 h 0.00 M 166.93 234.40 h 0.00 M 168.01 232.00 h 0.00 M 168.96 229.90 h 0.00 M 170.90 228.10 h 0.00 M 171.96 225.70 h 0.00 M 173.41 223.60 h 0.00 M 174.16 221.80 h 0.00 M 175.01 219.10 h 0.00 M 176.64 215.80 h 0.00 M 176.66 212.80 h 0.00 M 177.43 209.50 h 0.00 M 176.16 206.80 h 0.00 M 178.14 203.80 h 0.00 M 178.23 201.70 h 0.00 M 179.10 199.00 h 0.00 M 180.41 197.20 h 0.00 M 181.56 195.40 h 0.00 M 179.28 196.30 h 0.00 M 182.80 195.10 h 0.00 M 173.76 210.40 h 0.00 M 173.70 207.70 h 0.00 M 173.21 205.90 h 0.00 M 173.64 204.40 h 0.00 M 174.53 202.60 h 0.00 M 175.44 201.70 h 0.00 M 176.66 203.20 h 0.00 M 177.48 204.70 h 0.00 M 177.58 206.80 h 0.00 M 178.36 208.60 h 0.00 M 179.99 210.10 h 0.00 M 181.26 211.30 h 0.00 M 182.67 210.10 h 0.00 M 184.02 211.30 h 0.00 M 186.26 211.00 h 0.00 M 189.08 210.40 h 0.00 M 192.22 208.90 h 0.00 M 195.30 206.80 h 0.00 M 197.15 204.10 h 0.00 M 196.14 200.50 h 0.00 M 197.58 197.50 h 0.00 M 199.52 194.20 h 0.00 M 198.98 191.20 h 0.00 M 200.58 188.80 h 0.00 M 200.57 186.40 h 0.00 M 200.67 185.20 h 0.00 M 201.10 184.30 h 0.00 M 202.07 184.00 h 0.00 M 203.50 182.20 h 0.00 M 203.94 179.20 h 0.00 M 200.48 181.90 h 0.00 M 204.89 186.10 h 0.00 M 215.14 140.50 h 0.00 M 216.88 139.90 h 0.00 M 218.47 139.30 h 0.00 M 222.14 138.10 h 0.00 M 225.77 138.10 h 0.00 M 229.07 135.40 h 0.00 M 233.24 134.80 h 0.00 M 237.02 133.90 h 0.00 M 241.80 132.70 h 0.00 M 245.96 131.80 h 0.00 M 249.97 131.20 h 0.00 M 254.18 130.00 h 0.00 M 259.48 129.40 h 0.00 M 263.63 128.20 h 0.00 M 269.88 127.60 h 0.00 M 274.72 126.70 h 0.00 M 281.60 125.80 h 0.00 M 288.48 125.50 h 0.00 M 291.87 124.30 h 0.00 M 302.49 123.70 h 0.00 M 298.90 123.10 h 0.00 M 309.01 122.20 h 0.00 M 312.17 121.00 h 0.00 M 323.83 121.30 h 0.00 M 327.55 120.70 h 0.00 M 325.14 120.10 h 0.00 M 328.95 119.50 h 0.00 M 333.33 118.90 h 0.00 M 340.16 117.70 h 0.00 M 346.70 117.70 h 0.00 M 351.11 115.90 h 0.00 M 362.09 114.70 h 0.00 M 175.63 208.00 h 0.00 M 178.33 205.60 h 0.00 M 180.77 203.20 h 0.00 M 183.77 201.10 h 0.00 M 186.72 198.70 h 0.00 M 190.15 196.30 h 0.00 M 193.88 193.30 h 0.00 M 196.71 190.00 h 0.00 M 192.53 189.10 h 0.00 M 194.69 187.30 h 0.00 M 198.60 183.70 h 0.00 M 198.68 183.40 h 0.00 M 200.86 182.80 h 0.00 M 203.60 180.40 h 0.00 M 207.47 179.50 h 0.00 M 211.08 180.40 h 0.00 M 215.08 179.20 h 0.00 M 219.59 175.90 h 0.00 M 222.87 173.20 h 0.00 M 220.78 171.40 h 0.00 M 225.68 169.30 h 0.00 M 230.11 168.10 h 0.00 M 234.75 166.00 h 0.00 M 236.40 164.50 h 0.00 M 240.41 162.40 h 0.00 M 240.99 160.90 h 0.00 M 243.65 159.10 h 0.00 M 247.73 158.50 h 0.00 M 253.62 157.90 h 0.00 M 258.83 157.00 h 0.00 M 252.85 158.20 h 0.00 M 259.63 159.10 h 0.00 M 226.00 142.00 h 0.00 M 225.98 140.80 h 0.00 M 227.51 139.60 h 0.00 M 230.61 140.20 h 0.00 M 235.15 138.40 h 0.00 M 238.40 137.20 h 0.00 M 240.55 136.30 h 0.00 M 245.16 136.00 h 0.00 M 249.20 136.30 h 0.00 M 255.00 134.80 h 0.00 M 260.79 133.00 h 0.00 M 264.10 132.40 h 0.00 M 266.89 131.80 h 0.00 M 272.18 130.90 h 0.00 M 278.18 129.70 h 0.00 M 287.33 129.10 h 0.00 M 294.16 127.90 h 0.00 M 299.71 127.60 h 0.00 M 302.76 127.00 h 0.00 M 297.00 127.30 h 0.00 M 301.70 126.70 h 0.00 M 307.68 125.50 h 0.00 M 310.05 123.70 h 0.00 M 317.70 123.10 h 0.00 M 323.25 122.50 h 0.00 M 319.12 122.20 h 0.00 M 325.99 121.60 h 0.00 M 333.01 120.70 h 0.00 M 339.40 120.10 h 0.00 M 338.79 118.90 h 0.00 M 328.86 120.70 h 0.00 M 350.95 119.20 h 0.00 M 165.72 227.20 h 0.00 M 167.48 224.50 h 0.00 M 169.14 221.50 h 0.00 M 171.00 218.80 h 0.00 M 172.93 216.10 h 0.00 M 175.06 214.30 h 0.00 M 176.68 211.60 h 0.00 M 176.03 209.50 h 0.00 M 173.93 208.00 h 0.00 M 175.15 206.20 h 0.00 M 176.66 204.10 h 0.00 M 177.99 200.80 h 0.00 M 179.98 197.20 h 0.00 M 182.58 193.90 h 0.00 M 185.36 190.90 h 0.00 M 187.77 188.20 h 0.00 M 190.65 186.40 h 0.00 M 193.76 182.80 h 0.00 M 195.06 181.90 h 0.00 M 194.71 181.00 h 0.00 M 198.41 178.90 h 0.00 M 199.53 177.10 h 0.00 M 204.50 176.20 h 0.00 M 206.41 175.90 h 0.00 M 206.62 166.60 h 0.00 M 207.85 165.70 h 0.00 M 210.76 164.50 h 0.00 M 213.72 163.00 h 0.00 M 217.86 161.50 h 0.00 M 220.40 158.80 h 0.00 M 216.78 159.40 h 0.00 M 220.76 160.00 h 0.00 M 272.66 196.60 h 0.00 M 290.31 193.60 h 0.00 M 293.84 191.80 h 0.00 M 290.62 190.30 h 0.00 M 290.33 188.80 h 0.00 M 289.61 187.30 h 0.00 M 292.01 185.50 h 0.00 M 292.19 184.00 h 0.00 M 294.10 181.90 h 0.00 M 286.98 181.00 h 0.00 M 294.33 178.90 h 0.00 M 292.23 176.80 h 0.00 M 287.00 175.00 h 0.00 M 301.98 172.00 h 0.00 M 314.94 169.00 h 0.00 M 324.23 166.00 h 0.00 M 327.47 163.90 h 0.00 M 328.49 161.80 h 0.00 M 336.02 159.40 h 0.00 M 326.57 157.90 h 0.00 M 331.80 155.20 h 0.00 M 351.05 151.60 h 0.00 M 357.32 148.90 h 0.00 M 354.23 146.50 h 0.00 M 356.02 144.40 h 0.00 M 332.47 142.30 h 0.00 M 322.67 140.80 h 0.00 M 330.10 142.00 h 0.00 M 335.09 140.50 h 0.00 M 335.69 138.10 h 0.00 M 329.06 139.00 h 0.00 M 344.79 137.50 h 0.00 M 468.38 181.60 h 0.00 M 464.07 178.30 h 0.00 M 465.06 177.40 h 0.00 M 460.85 175.60 h 0.00 M 474.02 173.50 h 0.00 M 487.77 171.40 h 0.00 M 494.29 169.90 h 0.00 M 505.02 168.10 h 0.00 M 488.44 166.30 h 0.00 M 483.04 163.90 h 0.00 M 507.14 161.20 h 0.00 M 501.10 160.00 h 0.00 M 497.20 158.80 h 0.00 M 517.11 155.80 h 0.00 M 544.50 154.60 h 0.00 M 547.97 153.40 h 0.00 M 543.75 151.90 h 0.00 M 497.96 150.70 h 0.00 M 457.39 149.80 h 0.00 M 406.14 150.10 h 0.00 M 398.15 149.50 h 0.00 M 417.99 148.00 h 0.00 M 434.12 146.20 h 0.00 M 438.03 144.40 h 0.00 M 449.39 142.30 h 0.00 M 414.51 140.50 h 0.00 M 411.26 139.00 h 0.00 M 420.14 130.90 h 0.00 M 428.09 127.30 h 0.00 M 434.02 124.00 h 0.00 M 421.19 126.40 h 0.00 M 442.11 126.70 h 0.00 M 218.06 155.80 h 0.00 M 215.82 154.60 h 0.00 M 214.72 152.50 h 0.00 M 215.51 151.30 h 0.00 M 219.33 147.70 h 0.00 M 223.55 145.90 h 0.00 M 225.46 143.80 h 0.00 M 231.96 141.10 h 0.00 M 238.75 138.10 h 0.00 M 243.29 135.70 h 0.00 M 250.99 132.70 h 0.00 M 254.84 130.00 h 0.00 M 257.91 128.80 h 0.00 M 259.61 127.90 h 0.00 M 267.80 129.40 h 0.00 M 271.04 128.50 h 0.00 M 280.19 127.30 h 0.00 M 293.11 126.70 h 0.00 M 301.64 126.40 h 0.00 M 293.60 127.90 h 0.00 M 297.34 126.70 h 0.00 M 304.82 125.50 h 0.00 M 304.65 125.50 h 0.00 M 307.01 122.20 h 0.00 M 308.02 121.90 h 0.00 M 310.84 121.00 h 0.00 M 320.13 120.70 h 0.00 M 330.17 119.80 h 0.00 M 337.82 119.20 h 0.00 M 346.33 118.30 h 0.00 M 348.21 118.60 h 0.00 M 357.36 118.00 h 0.00 M 163.12 227.20 h 0.00 M 163.85 225.70 h 0.00 M 165.05 223.60 h 0.00 M 165.46 221.80 h 0.00 M 166.04 218.80 h 0.00 M 166.52 216.70 h 0.00 M 167.84 214.00 h 0.00 M 168.94 211.90 h 0.00 M 169.88 208.30 h 0.00 M 171.20 205.00 h 0.00 M 172.53 202.60 h 0.00 M 173.72 200.50 h 0.00 M 174.19 199.00 h 0.00 M 175.64 196.90 h 0.00 M 177.79 193.90 h 0.00 M 179.45 191.80 h 0.00 M 181.74 190.00 h 0.00 M 184.56 188.20 h 0.00 M 186.42 186.70 h 0.00 M 187.44 185.50 h 0.00 M 188.65 184.00 h 0.00 M 188.15 183.40 h 0.00 M 188.61 182.80 h 0.00 M 188.69 181.90 h 0.00 M 189.58 181.00 h 0.00 M 188.81 180.10 h 0.00 M 189.44 178.90 h 0.00 M 190.35 178.00 h 0.00 M 192.02 177.10 h 0.00 M 193.06 176.50 h 0.00 M 189.31 178.90 h 0.00 M 192.63 180.70 h 0.00 M 157.22 271.30 h 0.00 M 157.49 270.10 h 0.00 M 157.06 269.50 h 0.00 M 156.65 269.20 h 0.00 M 155.74 269.80 h 0.00 M 156.43 266.80 h 0.00 M 156.74 265.00 h 0.00 M 156.94 263.50 h 0.00 M 157.06 262.00 h 0.00 M 157.24 260.20 h 0.00 M 157.35 259.00 h 0.00 M 157.36 258.40 h 0.00 M 157.43 257.20 h 0.00 M 157.74 255.40 h 0.00 M 157.71 256.30 h 0.00 M 158.11 253.00 h 0.00 M 158.40 251.80 h 0.00 M 158.95 250.30 h 0.00 M 159.25 249.10 h 0.00 M 159.75 247.60 h 0.00 M 159.14 270.10 h 0.00 M 159.76 246.40 h 0.00 M 159.59 244.90 h 0.00 M 160.32 243.70 h 0.00 M 160.37 242.50 h 0.00 M 160.11 241.30 h 0.00 M 160.88 240.40 h 0.00 M 161.11 238.30 h 0.00 M 161.44 237.70 h 0.00 M 161.28 237.10 h 0.00 M 160.87 238.00 h 0.00 M 161.03 239.50 h 0.00 M 166.03 213.40 h 0.00 M 165.98 212.50 h 0.00 M 167.84 208.90 h 0.00 M 168.12 207.70 h 0.00 M 168.54 206.80 h 0.00 M 169.33 204.70 h 0.00 M 169.44 203.50 h 0.00 M 169.92 201.70 h 0.00 M 170.30 200.20 h 0.00 M 170.85 198.70 h 0.00 M 171.81 196.60 h 0.00 M 173.05 194.50 h 0.00 M 174.29 192.40 h 0.00 M 175.27 190.60 h 0.00 M 177.57 187.30 h 0.00 M 180.05 185.20 h 0.00 M 181.34 183.40 h 0.00 M 182.73 181.30 h 0.00 M 184.98 179.80 h 0.00 M 186.20 180.70 h 0.00 M 186.68 182.50 h 0.00 M 187.53 183.70 h 0.00 M 188.19 183.70 h 0.00 M 188.24 184.30 h 0.00 M 184.95 184.30 h 0.00 M 184.11 184.60 h 0.00 M 183.43 184.60 h 0.00 M 185.61 184.60 h 0.00 M 186.31 183.10 h 0.00 M 186.79 181.90 h 0.00 M 185.87 183.10 h 0.00 M 187.61 184.00 h 0.00 M 160.47 247.30 h 0.00 M 161.64 248.50 h 0.00 M 162.92 244.00 h 0.00 M 164.38 239.80 h 0.00 M 166.00 236.80 h 0.00 M 167.19 235.30 h 0.00 M 168.95 232.30 h 0.00 M 170.51 230.80 h 0.00 M 170.34 229.30 h 0.00 M 171.25 228.10 h 0.00 M 171.41 226.90 h 0.00 M 172.42 224.80 h 0.00 M 173.04 223.00 h 0.00 M 173.35 220.60 h 0.00 M 174.41 216.40 h 0.00 M 174.72 213.40 h 0.00 M 176.93 210.70 h 0.00 M 179.83 208.60 h 0.00 M 181.07 206.80 h 0.00 M 182.58 205.30 h 0.00 M 184.55 203.20 h 0.00 M 187.37 200.50 h 0.00 M 189.46 199.30 h 0.00 M 191.58 198.10 h 0.00 M 191.33 195.70 h 0.00 M 192.29 194.80 h 0.00 M 192.66 192.70 h 0.00 M 195.38 191.50 h 0.00 M 197.20 189.70 h 0.00 M 199.84 187.60 h 0.00 M 223.07 183.70 h 0.00 M 241.10 185.80 h 0.00 M 223.11 152.50 h 0.00 M 227.47 151.30 h 0.00 M 230.05 149.50 h 0.00 M 231.72 148.00 h 0.00 M 235.19 146.50 h 0.00 M 239.21 145.30 h 0.00 M 242.50 144.10 h 0.00 M 245.80 142.90 h 0.00 M 250.56 141.10 h 0.00 M 254.10 140.20 h 0.00 M 260.87 138.70 h 0.00 M 262.11 137.20 h 0.00 M 267.71 136.30 h 0.00 M 271.40 135.70 h 0.00 M 277.60 134.20 h 0.00 M 282.33 133.30 h 0.00 M 292.41 132.10 h 0.00 M 299.17 131.20 h 0.00 M 306.34 130.60 h 0.00 M 304.82 130.60 h 0.00 M 308.97 129.40 h 0.00 M 318.29 128.50 h 0.00 M 325.93 128.20 h 0.00 M 331.51 128.50 h 0.00 M 334.85 127.30 h 0.00 M 338.92 127.00 h 0.00 M 349.65 125.50 h 0.00 M 355.33 125.20 h 0.00 M 365.86 124.90 h 0.00 M 376.40 124.30 h 0.00 M 367.08 126.10 h 0.00 M 377.39 125.20 h 0.00 M 246.96 177.40 h 0.00 M 257.87 177.10 h 0.00 M 263.72 175.30 h 0.00 M 277.11 171.70 h 0.00 M 275.22 170.80 h 0.00 M 278.64 168.40 h 0.00 M 282.13 166.60 h 0.00 M 284.36 166.00 h 0.00 M 288.22 164.50 h 0.00 M 292.07 163.30 h 0.00 M 298.89 160.60 h 0.00 M 302.04 158.80 h 0.00 M 305.75 158.80 h 0.00 M 312.38 158.20 h 0.00 M 316.59 158.80 h 0.00 M 319.54 159.10 h 0.00 M 322.86 158.80 h 0.00 M 329.44 157.90 h 0.00 M 332.11 157.60 h 0.00 M 326.89 158.20 h 0.00 M 331.54 157.60 h 0.00 M 339.28 157.30 h 0.00 M 353.82 154.90 h 0.00 M 353.42 152.80 h 0.00 M 347.33 151.00 h 0.00 M 323.81 142.60 h 0.00 M 321.45 140.50 h 0.00 M 335.35 139.30 h 0.00 M 339.86 136.30 h 0.00 M 346.59 135.40 h 0.00 M 342.03 136.90 h 0.00 M 358.10 137.50 h 0.00 M 158.63 205.90 h 0.00 M 156.19 211.30 h 0.00 M 155.86 215.80 h 0.00 M 156.76 214.90 h 0.00 M 157.64 212.80 h 0.00 M 159.08 209.80 h 0.00 M 160.27 206.50 h 0.00 M 159.27 207.40 h 0.00 M 160.43 202.90 h 0.00 M 162.14 199.30 h 0.00 M 163.63 196.90 h 0.00 M 165.30 194.80 h 0.00 M 166.68 193.30 h 0.00 M 167.97 191.20 h 0.00 M 169.60 190.00 h 0.00 M 171.26 186.70 h 0.00 M 173.93 184.60 h 0.00 M 176.67 181.00 h 0.00 M 180.27 179.20 h 0.00 M 182.50 177.70 h 0.00 M 185.60 173.80 h 0.00 M 187.81 170.20 h 0.00 M 189.02 166.60 h 0.00 M 189.19 164.50 h 0.00 M 191.82 162.40 h 0.00 M 193.34 161.50 h 0.00 M 194.94 160.60 h 0.00 M 197.58 159.40 h 0.00 M 200.35 158.20 h 0.00 M 203.81 157.00 h 0.00 M 202.50 161.80 h 0.00 M 208.10 161.20 h 0.00 M 180.40 198.10 h 0.00 M 177.70 199.60 h 0.00 M 176.81 199.90 h 0.00 M 174.94 201.10 h 0.00 M 172.49 202.90 h 0.00 M 171.47 203.50 h 0.00 M 172.35 202.30 h 0.00 M 173.54 201.10 h 0.00 M 173.77 200.20 h 0.00 M 175.05 198.70 h 0.00 M 178.35 196.00 h 0.00 M 183.14 191.50 h 0.00 M 187.07 188.20 h 0.00 M 191.30 184.60 h 0.00 M 196.29 181.00 h 0.00 M 202.04 178.30 h 0.00 M 208.90 174.40 h 0.00 M 215.24 172.60 h 0.00 M 217.44 172.60 h 0.00 M 217.34 171.70 h 0.00 M 222.18 169.90 h 0.00 M 228.09 167.50 h 0.00 M 232.86 165.10 h 0.00 M 242.28 162.40 h 0.00 M 243.13 160.30 h 0.00 M 237.41 158.50 h 0.00 M 239.67 158.50 h 0.00 M 243.65 156.70 h 0.00 M 248.59 155.80 h 0.00 M 253.58 154.30 h 0.00 M 250.96 155.80 h 0.00 M 258.70 156.70 h 0.00 M 150.49 300.10 h 0.00 M 150.54 297.10 h 0.00 M 150.62 293.20 h 0.00 M 150.77 291.10 h 0.00 M 150.92 289.30 h 0.00 M 151.08 287.80 h 0.00 M 151.23 285.70 h 0.00 M 151.39 284.50 h 0.00 M 151.53 282.40 h 0.00 M 151.75 279.70 h 0.00 M 152.14 277.00 h 0.00 M 152.59 274.00 h 0.00 M 153.04 271.00 h 0.00 M 153.62 268.00 h 0.00 M 154.36 265.00 h 0.00 M 155.24 262.00 h 0.00 M 156.25 259.00 h 0.00 M 157.33 256.00 h 0.00 M 158.38 260.50 h 0.00 M 159.37 249.70 h 0.00 M 160.47 247.00 h 0.00 M 161.50 243.70 h 0.00 M 162.75 240.70 h 0.00 M 163.67 237.10 h 0.00 M 164.56 234.10 h 0.00 M 164.92 231.40 h 0.00 M 164.62 228.40 h 0.00 M 165.35 226.00 h 0.00 M 166.69 223.00 h 0.00 M 168.10 220.60 h 0.00 M 168.82 220.00 h 0.00 M 165.87 224.50 h 0.00 M 161.57 254.80 h 0.00 M 162.11 253.60 h 0.00 M 162.70 252.40 h 0.00 M 163.21 250.90 h 0.00 M 163.73 249.70 h 0.00 M 164.41 247.60 h 0.00 M 164.77 245.20 h 0.00 M 165.36 242.50 h 0.00 M 165.96 239.80 h 0.00 M 166.45 237.40 h 0.00 M 167.05 235.00 h 0.00 M 167.44 233.20 h 0.00 M 168.00 231.70 h 0.00 M 168.45 230.20 h 0.00 M 169.17 228.70 h 0.00 M 170.04 227.80 h 0.00 M 171.40 225.70 h 0.00 M 172.99 223.00 h 0.00 M 173.78 221.20 h 0.00 M 173.59 220.00 h 0.00 M 174.14 218.50 h 0.00 M 175.25 216.70 h 0.00 M 176.37 215.20 h 0.00 M 177.29 214.00 h 0.00 M 178.48 208.90 h 0.00 M 180.14 208.30 h 0.00 M 180.24 208.30 h 0.00 M 180.63 208.00 h 0.00 M 181.96 208.00 h 0.00 M 183.36 207.40 h 0.00 M 182.67 209.50 h 0.00 M 186.45 211.90 h 0.00 M 180.21 201.40 h 0.00 M 182.02 200.80 h 0.00 M 183.43 199.90 h 0.00 M 182.27 199.30 h 0.00 M 183.99 197.80 h 0.00 M 181.86 197.80 h 0.00 M 184.10 195.70 h 0.00 M 186.50 193.60 h 0.00 M 188.22 191.20 h 0.00 M 189.20 189.40 h 0.00 M 191.61 187.30 h 0.00 M 191.73 186.40 h 0.00 M 191.87 184.30 h 0.00 M 192.97 182.50 h 0.00 M 195.24 181.00 h 0.00 M 197.40 180.10 h 0.00 M 201.88 177.70 h 0.00 M 203.83 177.10 h 0.00 M 206.37 176.50 h 0.00 M 205.05 176.50 h 0.00 M 207.85 176.20 h 0.00 M 212.71 174.40 h 0.00 M 215.17 172.00 h 0.00 M 215.98 172.00 h 0.00 M 218.64 170.80 h 0.00 M 219.64 169.60 h 0.00 M 223.64 168.40 h 0.00 M 225.24 167.50 h 0.00 M 226.83 166.90 h 0.00 M 225.98 166.30 h 0.00 M 219.49 173.20 h 0.00 M 223.51 172.60 h 0.00 M 153.11 277.00 h 0.00 M 153.10 275.80 h 0.00 M 152.91 278.20 h 0.00 M 152.41 280.60 h 0.00 M 153.01 278.80 h 0.00 M 153.22 274.30 h 0.00 M 153.51 271.30 h 0.00 M 154.09 268.60 h 0.00 M 153.88 268.30 h 0.00 M 153.93 267.10 h 0.00 M 153.86 266.20 h 0.00 M 153.88 265.30 h 0.00 M 154.01 263.80 h 0.00 M 154.32 262.00 h 0.00 M 154.27 262.30 h 0.00 M 154.05 262.60 h 0.00 M 154.19 260.80 h 0.00 M 154.12 262.90 h 0.00 M 154.28 260.80 h 0.00 M 154.46 258.40 h 0.00 M 154.71 256.90 h 0.00 M 155.04 253.90 h 0.00 M 155.13 252.70 h 0.00 M 155.38 250.30 h 0.00 M 155.65 248.50 h 0.00 M 156.01 245.80 h 0.00 M 156.22 244.60 h 0.00 M 156.43 242.80 h 0.00 M 156.81 241.60 h 0.00 M 157.15 239.50 h 0.00 M 157.22 239.50 h 0.00 M 157.90 238.30 h 0.00 M 151.81 328.90 h 0.00 M 152.19 326.50 h 0.00 M 152.09 325.30 h 0.00 M 152.20 322.90 h 0.00 M 152.33 320.80 h 0.00 M 152.36 319.00 h 0.00 M 152.59 316.90 h 0.00 M 152.75 313.30 h 0.00 M 152.98 310.30 h 0.00 M 153.15 306.70 h 0.00 M 153.12 304.90 h 0.00 M 153.73 301.30 h 0.00 M 153.80 298.60 h 0.00 M 154.18 295.00 h 0.00 M 154.23 292.90 h 0.00 M 154.57 290.20 h 0.00 M 154.81 287.50 h 0.00 M 154.98 285.40 h 0.00 M 155.18 283.00 h 0.00 M 155.31 280.90 h 0.00 M 155.52 278.80 h 0.00 M 155.65 277.30 h 0.00 M 155.42 277.90 h 0.00 M 155.60 277.90 h 0.00 M 155.96 275.50 h 0.00 M 156.37 275.20 h 0.00 M 156.93 273.70 h 0.00 M 157.12 272.20 h 0.00 M 157.44 271.00 h 0.00 M 157.73 270.10 h 0.00 M 157.46 271.90 h 0.00 M 157.83 271.60 h 0.00 M 182.63 181.00 h 0.00 M 184.96 180.10 h 0.00 M 187.12 178.90 h 0.00 M 189.37 175.90 h 0.00 M 192.16 174.70 h 0.00 M 195.54 172.60 h 0.00 M 198.17 173.20 h 0.00 M 203.36 170.20 h 0.00 M 206.26 167.20 h 0.00 M 210.08 166.00 h 0.00 M 223.23 166.30 h 0.00 M 224.06 161.20 h 0.00 M 227.32 160.00 h 0.00 M 229.27 156.40 h 0.00 M 231.26 152.50 h 0.00 M 233.79 150.40 h 0.00 M 237.76 150.70 h 0.00 M 244.41 148.90 h 0.00 M 249.98 147.70 h 0.00 M 250.46 145.90 h 0.00 M 258.36 141.70 h 0.00 M 259.31 142.90 h 0.00 M 264.15 140.80 h 0.00 M 271.97 138.40 h 0.00 M 279.77 135.70 h 0.00 M 291.63 133.30 h 0.00 M 300.20 132.70 h 0.00 M 314.69 129.70 h 0.00 M 322.50 127.00 h 0.00 M 336.46 125.50 h 0.00 M 320.09 126.70 h 0.00 M 339.33 124.60 h 0.00 M 174.82 202.00 h 0.00 M 176.32 200.50 h 0.00 M 179.55 201.10 h 0.00 M 179.87 199.90 h 0.00 M 181.47 199.00 h 0.00 M 183.25 196.60 h 0.00 M 186.21 194.80 h 0.00 M 189.56 193.00 h 0.00 M 193.21 190.00 h 0.00 M 197.27 189.10 h 0.00 M 201.58 186.40 h 0.00 M 204.74 183.70 h 0.00 M 209.78 180.40 h 0.00 M 219.54 178.90 h 0.00 M 226.76 174.40 h 0.00 M 233.70 172.60 h 0.00 M 247.29 169.90 h 0.00 M 254.15 168.10 h 0.00 M 259.17 166.90 h 0.00 M 254.37 163.90 h 0.00 M 258.47 163.00 h 0.00 M 259.66 161.80 h 0.00 M 262.29 160.00 h 0.00 M 261.62 158.20 h 0.00 M 262.39 157.30 h 0.00 M 254.31 155.20 h 0.00 M 247.86 155.50 h 0.00 M 248.80 154.90 h 0.00 M 248.55 155.50 h 0.00 M 249.43 153.70 h 0.00 M 243.03 154.60 h 0.00 M 245.77 157.00 h 0.00 M 239.65 181.90 h 0.00 M 246.18 178.90 h 0.00 M 251.87 175.60 h 0.00 M 263.45 172.60 h 0.00 M 274.86 169.00 h 0.00 M 282.61 166.30 h 0.00 M 289.40 163.30 h 0.00 M 298.59 159.40 h 0.00 M 292.02 157.00 h 0.00 M 301.11 154.00 h 0.00 M 315.71 150.70 h 0.00 M 313.15 150.10 h 0.00 M 320.68 148.00 h 0.00 M 334.76 146.50 h 0.00 M 356.11 144.70 h 0.00 M 373.01 137.50 h 0.00 M 392.98 135.70 h 0.00 M 411.03 133.90 h 0.00 M 406.94 132.40 h 0.00 M 401.20 132.40 h 0.00 M 436.17 127.00 h 0.00 M 453.93 125.50 h 0.00 M 461.60 124.00 h 0.00 M 465.17 123.10 h 0.00 M 470.59 121.60 h 0.00 M 480.31 121.00 h 0.00 M 489.91 119.80 h 0.00 M 512.16 119.50 h 0.00 M 532.39 118.00 h 0.00 M 539.96 117.10 h 0.00 M 530.80 118.30 h 0.00 M 601.90 118.30 h 0.00 M 166.18 222.10 h 0.00 M 168.40 220.00 h 0.00 M 170.79 217.60 h 0.00 M 172.23 215.80 h 0.00 M 172.26 214.60 h 0.00 M 172.30 214.00 h 0.00 M 172.25 213.10 h 0.00 M 172.68 212.50 h 0.00 M 173.03 212.20 h 0.00 M 174.66 210.10 h 0.00 M 177.72 208.00 h 0.00 M 178.89 207.10 h 0.00 M 179.93 204.40 h 0.00 M 182.45 202.00 h 0.00 M 183.86 199.90 h 0.00 M 184.67 200.80 h 0.00 M 186.35 196.00 h 0.00 M 187.49 194.50 h 0.00 M 186.55 192.10 h 0.00 M 185.60 189.70 h 0.00 M 185.59 187.90 h 0.00 M 185.46 187.30 h 0.00 M 183.64 186.10 h 0.00 M 184.41 186.40 h 0.00 M 184.15 187.30 h 0.00 M 184.42 187.60 h 0.00 M 183.82 186.40 h 0.00 M 182.73 187.90 h 0.00 M 183.25 188.20 h 0.00 M 184.78 188.50 h 0.00 M 179.91 193.00 h 0.00 M 185.76 195.10 h 0.00 M 163.84 207.40 h 0.00 M 163.85 205.90 h 0.00 M 165.08 204.70 h 0.00 M 165.76 203.80 h 0.00 M 166.96 202.00 h 0.00 M 167.77 200.20 h 0.00 M 168.93 199.30 h 0.00 M 168.62 199.00 h 0.00 M 168.87 198.40 h 0.00 M 170.68 196.30 h 0.00 M 170.56 195.70 h 0.00 M 171.25 194.20 h 0.00 M 172.09 193.90 h 0.00 M 172.55 192.40 h 0.00 M 174.22 188.80 h 0.00 M 174.95 189.40 h 0.00 M 176.00 188.50 h 0.00 M 176.26 188.20 h 0.00 M 176.88 187.30 h 0.00 M 176.53 186.70 h 0.00 M 177.56 185.80 h 0.00 M 178.84 184.30 h 0.00 M 180.90 183.70 h 0.00 M 183.35 182.20 h 0.00 M 189.83 181.30 h 0.00 M 193.71 181.30 h 0.00 M 194.64 179.50 h 0.00 M 197.77 177.70 h 0.00 M 200.85 176.50 h 0.00 M 201.51 176.20 h 0.00 M 193.04 178.90 h 0.00 M 192.47 181.00 h 0.00 M 193.80 189.70 h 0.00 M 197.46 186.70 h 0.00 M 199.15 181.90 h 0.00 M 199.22 177.70 h 0.00 M 200.65 173.80 h 0.00 M 203.81 171.70 h 0.00 M 205.77 169.30 h 0.00 M 209.07 166.60 h 0.00 M 212.50 164.20 h 0.00 M 216.43 164.50 h 0.00 M 220.85 162.70 h 0.00 M 223.32 160.90 h 0.00 M 226.48 160.60 h 0.00 M 228.37 159.40 h 0.00 M 230.75 158.50 h 0.00 M 235.50 158.20 h 0.00 M 242.96 156.70 h 0.00 M 246.98 154.60 h 0.00 M 250.52 153.40 h 0.00 M 249.73 152.50 h 0.00 M 252.79 151.30 h 0.00 M 250.91 149.50 h 0.00 M 249.65 149.20 h 0.00 M 255.36 146.50 h 0.00 M 258.43 145.60 h 0.00 M 261.93 145.00 h 0.00 M 269.35 144.10 h 0.00 M 274.82 142.30 h 0.00 M 282.00 142.00 h 0.00 M 291.03 139.90 h 0.00 M 282.11 141.10 h 0.00 M 288.52 140.20 h 0.00 M 160.61 242.20 h 0.00 M 161.61 240.40 h 0.00 M 162.50 238.00 h 0.00 M 163.48 235.60 h 0.00 M 164.66 232.60 h 0.00 M 166.02 229.30 h 0.00 M 167.40 226.60 h 0.00 M 168.29 223.30 h 0.00 M 165.67 224.50 h 0.00 M 165.79 223.00 h 0.00 M 166.76 221.50 h 0.00 M 167.56 218.80 h 0.00 M 168.43 216.40 h 0.00 M 169.49 213.70 h 0.00 M 170.82 214.30 h 0.00 M 172.50 210.40 h 0.00 M 174.23 208.30 h 0.00 M 176.20 207.10 h 0.00 M 178.02 206.20 h 0.00 M 179.17 202.90 h 0.00 M 181.04 200.80 h 0.00 M 183.30 198.70 h 0.00 M 185.67 196.60 h 0.00 M 187.05 195.10 h 0.00 M 187.91 193.90 h 0.00 M 187.85 191.50 h 0.00 M 188.86 190.30 h 0.00 M 190.61 188.80 h 0.00 M 193.39 187.00 h 0.00 M 196.01 185.20 h 0.00 M 195.25 187.30 h 0.00 M 198.78 188.50 h 0.00 M 161.69 213.70 h 0.00 M 162.13 212.50 h 0.00 M 162.09 212.50 h 0.00 M 162.83 211.30 h 0.00 M 164.56 208.30 h 0.00 M 165.74 205.90 h 0.00 M 166.21 204.70 h 0.00 M 167.31 203.20 h 0.00 M 167.12 200.80 h 0.00 M 167.34 198.10 h 0.00 M 167.97 197.20 h 0.00 M 168.26 194.80 h 0.00 M 169.39 194.50 h 0.00 M 170.44 194.80 h 0.00 M 171.88 193.00 h 0.00 M 173.87 194.20 h 0.00 M 176.34 192.70 h 0.00 M 179.25 190.00 h 0.00 M 182.41 187.60 h 0.00 M 182.76 186.70 h 0.00 M 185.74 182.50 h 0.00 M 188.55 180.40 h 0.00 M 189.61 177.40 h 0.00 M 191.58 175.00 h 0.00 M 192.39 173.50 h 0.00 M 192.66 172.30 h 0.00 M 194.39 170.50 h 0.00 M 196.33 169.00 h 0.00 M 198.46 167.20 h 0.00 M 199.50 166.00 h 0.00 M 193.93 171.40 h 0.00 M 201.61 171.40 h 0.00 M 166.91 204.70 h 0.00 M 167.89 204.40 h 0.00 M 168.32 202.60 h 0.00 M 168.78 201.40 h 0.00 M 169.66 199.90 h 0.00 M 170.17 198.70 h 0.00 M 170.54 197.80 h 0.00 M 171.46 196.30 h 0.00 M 172.05 194.80 h 0.00 M 170.88 195.10 h 0.00 M 171.24 193.90 h 0.00 M 172.25 192.10 h 0.00 M 173.16 190.60 h 0.00 M 173.87 189.10 h 0.00 M 176.17 187.00 h 0.00 M 178.00 185.50 h 0.00 M 179.69 184.00 h 0.00 M 180.66 183.10 h 0.00 M 182.74 181.30 h 0.00 M 182.58 180.70 h 0.00 M 183.61 179.20 h 0.00 M 186.49 177.10 h 0.00 M 188.18 174.70 h 0.00 M 191.37 173.50 h 0.00 M 193.53 172.00 h 0.00 M 191.10 170.50 h 0.00 M 191.09 171.40 h 0.00 M 193.42 171.40 h 0.00 M 194.21 171.40 h 0.00 M 194.16 172.00 h 0.00 M 190.59 180.70 h 0.00 M 193.77 178.00 h 0.00\"/>
	</g>
	<text class=\"poloto_text poloto_name poloto_title\" x=\"400.00\" y=\"37.50\">HDI vs GDP</text>
	<text class=\"poloto_text poloto_name poloto_x\" x=\"400.00\" y=\"481.25\">HDI</text>
	<text class=\"poloto_text poloto_name poloto_y\" x=\"37.50\" y=\"250.00\" transform=\"rotate(-90,37.50,250.00)\">GDP</text>
	<text class=\"poloto_text poloto_ticks poloto_y\">
		<tspan x=\"135.00\" y=\"400.00\">0.0</tspan>
		<tspan x=\"135.00\" y=\"340.00\">0.2</tspan>
		<tspan x=\"135.00\" y=\"280.00\">0.4</tspan>
		<tspan x=\"135.00\" y=\"220.00\">0.6</tspan>
		<tspan x=\"135.00\" y=\"160.00\">0.8</tspan>
		<tspan x=\"135.00\" y=\"100.00\">1.0</tspan>
	</text>
	<g class=\"poloto_imgs poloto_ticks poloto_y\" stroke=\"black\">
		<line x1=\"150.00\" x2=\"144.00\" y1=\"400.00\" y2=\"400.00\"/>
		<line x1=\"150.00\" x2=\"144.00\" y1=\"340.00\" y2=\"340.00\"/>
		<line x1=\"150.00\" x2=\"144.00\" y1=\"280.00\" y2=\"280.00\"/>
		<line x1=\"150.00\" x2=\"144.00\" y1=\"220.00\" y2=\"220.00\"/>
		<line x1=\"150.00\" x2=\"144.00\" y1=\"160.00\" y2=\"160.00\"/>
		<line x1=\"150.00\" x2=\"144.00\" y1=\"100.00\" y2=\"100.00\"/>
	</g>
	<text class=\"poloto_text poloto_ticks poloto_x\">
		<tspan x=\"225.12\" y=\"430.00\">20000</tspan>
		<tspan x=\"301.32\" y=\"430.00\">40000</tspan>
		<tspan x=\"377.53\" y=\"430.00\">60000</tspan>
		<tspan x=\"453.73\" y=\"430.00\">80000</tspan>
		<tspan x=\"529.94\" y=\"430.00\">1e5</tspan>
		<tspan x=\"606.14\" y=\"430.00\">1.2e5</tspan>
	</text>
	<g class=\"poloto_imgs poloto_ticks poloto_x\" stroke=\"black\">
		<line x1=\"225.12\" x2=\"225.12\" y1=\"400.00\" y2=\"405.00\"/>
		<line x1=\"301.32\" x2=\"301.32\" y1=\"400.00\" y2=\"405.00\"/>
		<line x1=\"377.53\" x2=\"377.53\" y1=\"400.00\" y2=\"405.00\"/>
		<line x1=\"453.73\" x2=\"453.73\" y1=\"400.00\" y2=\"405.00\"/>
		<line x1=\"529.94\" x2=\"529.94\" y1=\"400.00\" y2=\"405.00\"/>
		<line x1=\"606.14\" x2=\"606.14\" y1=\"400.00\" y2=\"405.00\"/>
	</g>
	<path class=\"poloto_imgs poloto_ticks poloto_x\" stroke=\"black\" style=\"stroke-dasharray:9.525495465123369;stroke-dashoffset:-75.120913280781;\" d=\" M 150.00 400.00 L 650.00 400.00\"/>
	<path class=\"poloto_imgs poloto_ticks poloto_y\" stroke=\"black\" style=\"stroke-dasharray:7.5;stroke-dashoffset:-0;\" d=\" M 150.00 400.00 L 150.00 100.00\"/>
</svg>
</div>

"}


" \n\t\n\t \n\t \n\t\t<path d=\" M 177.93 181.30 h 0.00 M 176.24 183.10 h 0.00 M 174.02 185.50 h 0.00 M 170.96 189.10 h 0.00 M 166.36 193.30 h 0.00 M 164.68 194.80 h 0.00 M 163.49 194.50 h 0.00 M 163.43 193.30 h 0.00 M 163.44 191.20 h 0.00 M 163.76 190.60 h 0.00 M 165.15 190.00 h 0.00 M 167.17 185.50 h 0.00 M 168.63 183.40 h 0.00 M 171.11 180.40 h 0.00 M 174.59 178.00 h 0.00 M 176.40 177.10 h 0.00 M 179.60 174.70 h 0.00 M 183.23 172.90 h 0.00 M 184.86 171.70 h 0.00 M 179.76 172.60 h 0.00 M 181.53 170.80 h 0.00 M 184.15 169.30 h 0.00 M 185.90 168.10 h 0.00 M 191.25 168.10 h 0.00 M 189.85 168.10 h 0.00 M 187.65 167.80 h 0.00 M 191.39 166.30 h 0.00 M 194.11 165.40 h 0.00 M 197.05 165.10 h 0.00 M 199.78 164.20 h 0.00 M 198.84 167.50 h 0.00 M 203.36 168.10 h 0.00 M 151.85 256.30 h 0.00 M 152.10 256.60 h 0.00 M 152.35 257.20 h 0.00 M 152.50 258.40 h 0.00 M 152.74 257.50 h 0.00 M 152.90 258.40 h 0.00 M 153.16 259.90 h 0.00 M 153.35 261.40 h 0.00 M 153.44 262.00 h 0.00 M 153.50 264.40 h 0.00 M 153.78 264.40 h 0.00 M 154.06 265.60 h 0.00 M 154.19 266.20 h 0.00 M 154.55 265.30 h 0.00 M 154.82 265.60 h 0.00 M 155.24 265.60 h 0.00 M 155.72 266.20 h 0.00 M 156.17 264.40 h 0.00 M 156.67 262.90 h 0.00 M 156.57 262.90 h 0.00 M 157.01 259.90 h 0.00 M 157.50 257.50 h 0.00 M 157.54 254.20 h 0.00 M 158.47 253.00 h 0.00 M 159.34 251.20 h 0.00 M 160.31 249.10 h 0.00 M 159.95 245.80 h 0.00 M 158.72 244.60 h 0.00 M 158.68 243.40 h 0.00 M 158.66 242.80 h 0.00 M 158.11 243.70 h 0.00 M 158.56 245.80 h 0.00 M 223.08 151.30 h 0.00 M 228.92 149.50 h 0.00 M 231.68 147.40 h 0.00 M 232.27 144.40 h 0.00 M 235.78 142.60 h 0.00 M 238.71 141.10 h 0.00 M 240.66 139.60 h 0.00 M 242.54 137.80 h 0.00 M 245.49 136.30 h 0.00 M 249.09 134.80 h 0.00 M 252.57 133.30 h 0.00 M 257.03 131.20 h 0.00 M 260.09 129.70 h 0.00 M 262.88 128.50 h 0.00 M 268.31 125.80 h 0.00 M 270.06 125.80 h 0.00 M 279.07 124.90 h 0.00 M 287.20 123.70 h 0.00 M 292.95 123.10 h 0.00 M 289.19 123.10 h 0.00 M 297.33 122.20 h 0.00 M 311.01 120.70 h 0.00 M 314.13 120.10 h 0.00 M 320.35 119.80 h 0.00 M 328.04 118.90 h 0.00 M 330.32 118.60 h 0.00 M 341.64 117.70 h 0.00 M 351.13 116.80 h 0.00 M 359.22 116.50 h 0.00 M 367.61 115.60 h 0.00 M 364.13 116.80 h 0.00 M 372.95 117.40 h 0.00 M 263.03 164.20 h 0.00 M 275.39 161.20 h 0.00 M 278.88 160.00 h 0.00 M 285.70 157.00 h 0.00 M 292.00 154.30 h 0.00 M 295.03 152.80 h 0.00 M 300.26 150.40 h 0.00 M 310.80 148.30 h 0.00 M 317.59 146.20 h 0.00 M 338.84 141.70 h 0.00 M 358.97 140.80 h 0.00 M 362.81 139.30 h 0.00 M 372.10 137.80 h 0.00 M 378.29 138.10 h 0.00 M 395.41 135.40 h 0.00 M 411.01 134.80 h 0.00 M 449.44 134.50 h 0.00 M 473.24 133.90 h 0.00 M 495.53 130.00 h 0.00 M 479.93 128.50 h 0.00 M 493.20 126.40 h 0.00 M 508.89 126.10 h 0.00 M 517.12 125.20 h 0.00 M 533.46 123.70 h 0.00 M 550.12 122.20 h 0.00 M 559.88 125.50 h 0.00 M 580.86 123.10 h 0.00 M 587.04 124.30 h 0.00 M 592.80 123.40 h 0.00 M 604.39 121.90 h 0.00 M 606.18 122.80 h 0.00 M 650.00 121.00 h 0.00 M 169.04 189.10 h 0.00 M 167.19 193.30 h 0.00 M 166.12 196.30 h 0.00 M 166.82 196.00 h 0.00 M 167.95 195.40 h 0.00 M 169.60 193.60 h 0.00 M 170.81 192.10 h 0.00 M 170.12 191.20 h 0.00 M 170.04 190.00 h 0.00 M 170.24 187.60 h 0.00 M 171.20 185.50 h 0.00 M 173.76 183.40 h 0.00 M 176.21 181.30 h 0.00 M 177.72 180.40 h 0.00 M 183.17 177.40 h 0.00 M 185.50 173.20 h 0.00 M 192.94 169.00 h 0.00 M 201.12 164.20 h 0.00 M 212.86 158.20 h 0.00 M 212.29 157.30 h 0.00 M 215.05 157.90 h 0.00 M 220.56 157.60 h 0.00 M 224.39 158.50 h 0.00 M 223.89 157.00 h 0.00 M 227.53 156.70 h 0.00 M 231.31 156.10 h 0.00 M 240.00 155.20 h 0.00 M 251.58 153.10 h 0.00 M 261.58 151.90 h 0.00 M 274.91 150.40 h 0.00 M 275.93 152.80 h 0.00 M 287.14 153.70 h 0.00 M 175.49 222.70 h 0.00 M 175.42 221.80 h 0.00 M 175.88 220.30 h 0.00 M 175.34 219.70 h 0.00 M 175.10 219.40 h 0.00 M 176.14 218.20 h 0.00 M 177.27 214.90 h 0.00 M 177.59 212.80 h 0.00 M 178.93 210.10 h 0.00 M 179.90 207.70 h 0.00 M 181.34 205.30 h 0.00 M 182.60 203.50 h 0.00 M 184.56 20